# SymmetryInspiredFeatureMap: Complete Feature Demonstration

**Library:** [encoding-atlas](https://pypi.org/project/encoding-atlas/0.2.0/) v0.2.0
**Encoding:** `SymmetryInspiredFeatureMap` — A symmetry-aware quantum data encoding

This notebook is the **definitive reference** for `SymmetryInspiredFeatureMap`. It covers every constructor parameter, every public method, all four symmetry types, all three feature maps, all four entanglement patterns, cross-backend circuit generation (PennyLane / Qiskit / Cirq), analysis tools, edge cases, serialization, thread safety, and more.

**What is SymmetryInspiredFeatureMap?**

A heuristic quantum data encoding that incorporates symmetry-aware circuit structures inspired by geometric quantum machine learning. It provides useful inductive bias for problems with underlying symmetries (rotational patterns in images, molecular spatial symmetries, periodic time-series data) without the implementation complexity of mathematically rigorous equivariant constructions.

**Circuit structure (per repetition):**
1. **Encoding layer** — Hadamard + RY(feature-mapped angle) on each qubit
2. **Equivariant layer** — RZ(symmetry-dependent angle) on each qubit
3. **Entanglement layer** — Symmetry-specific two-qubit gates on qubit pairs

> **Important:** This is a *heuristic* encoding — it does NOT satisfy U(g)|ψ(x)⟩ = |ψ(g·x)⟩. The symmetry bias comes from circuit topology and parameter choices, not algebraic invariance.

---
## 1. Installation & Setup

In [1]:
# Install the library (uncomment if not already installed)
# !pip install encoding-atlas

# For full multi-backend support:
# !pip install encoding-atlas[qiskit,cirq]

In [2]:
import numpy as np
import encoding_atlas

print(f"encoding-atlas version: {encoding_atlas.__version__}")
print(f"NumPy version: {np.__version__}")

encoding-atlas version: 0.2.0
NumPy version: 2.2.6


In [3]:
# Check which quantum backends are available
backends_available = {}

try:
    import pennylane as qml
    backends_available['pennylane'] = qml.__version__
except ImportError:
    pass

try:
    import qiskit
    backends_available['qiskit'] = qiskit.__version__
except ImportError:
    pass

try:
    import cirq
    backends_available['cirq'] = cirq.__version__
except ImportError:
    pass

print("Available backends:")
for name, version in backends_available.items():
    print(f"  {name}: {version}")

assert len(backends_available) >= 1, "At least one quantum backend is required!" 

Available backends:
  pennylane: 0.42.3
  qiskit: 2.3.0
  cirq: 1.5.0


---
## 2. Creating a SymmetryInspiredFeatureMap

The constructor accepts six parameters:

| Parameter | Type | Default | Valid Values | Notes |
|---|---|---|---|---|
| `n_features` | `int` | *(required)* | ≥ 2 | Must be even for `rotation` symmetry |
| `symmetry` | `str` | `'rotation'` | `rotation`, `cyclic`, `reflection`, `full` | Case-insensitive |
| `reps` | `int` | `2` | ≥ 1 | Number of circuit repetitions |
| `entanglement` | `str` | `'linear'` | `full`, `linear`, `circular`, `none` | Case-insensitive |
| `feature_map` | `str` | `'angle'` | `angle`, `fourier`, `polynomial` | How features map to angles |
| `include_barriers` | `bool` | `True` | `True` / `False` | Qiskit visualization aid |

In [4]:
from encoding_atlas import SymmetryInspiredFeatureMap

# Basic creation with all defaults
enc_default = SymmetryInspiredFeatureMap(n_features=4)
print(f"Default: {enc_default}")
print(f"  symmetry      = {enc_default.symmetry!r}")
print(f"  reps          = {enc_default.reps}")
print(f"  entanglement  = {enc_default.entanglement!r}")
print(f"  feature_map   = {enc_default.feature_map!r}")
print(f"  barriers      = {enc_default.include_barriers}")

Default: SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2, entanglement='linear', feature_map='angle', include_barriers=True)
  symmetry      = 'rotation'
  reps          = 2
  entanglement  = 'linear'
  feature_map   = 'angle'
  barriers      = True


In [5]:
# All four symmetry types
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym)
    print(f"symmetry={sym!r:14s}  n_qubits={enc.n_qubits}  depth={enc.depth}")

symmetry='rotation'      n_qubits=4  depth=8
symmetry='cyclic'        n_qubits=4  depth=10
symmetry='reflection'    n_qubits=4  depth=10
symmetry='full'          n_qubits=4  depth=10


In [6]:
# Case insensitivity — all of these are equivalent
enc_upper = SymmetryInspiredFeatureMap(n_features=4, symmetry='ROTATION')
enc_mixed = SymmetryInspiredFeatureMap(n_features=4, symmetry='Rotation')
enc_lower = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation')

assert enc_upper == enc_mixed == enc_lower
print("Case-insensitive: ROTATION == Rotation == rotation ✓")

# Same for entanglement and feature_map
enc_ent = SymmetryInspiredFeatureMap(n_features=4, entanglement='LINEAR')
assert enc_ent.entanglement == 'linear'
print("Case-insensitive entanglement ✓")

enc_fm = SymmetryInspiredFeatureMap(n_features=4, feature_map='FOURIER')
assert enc_fm.feature_map == 'fourier'
print("Case-insensitive feature_map ✓")

Case-insensitive: ROTATION == Rotation == rotation ✓
Case-insensitive entanglement ✓
Case-insensitive feature_map ✓


In [7]:
# Custom combination of all parameters
enc_custom = SymmetryInspiredFeatureMap(
    n_features=6,
    symmetry='reflection',
    reps=3,
    entanglement='circular',
    feature_map='polynomial',
    include_barriers=False,
)
print(f"Custom: {enc_custom}")

Custom: SymmetryInspiredFeatureMap(n_features=6, symmetry='reflection', reps=3, entanglement='circular', feature_map='polynomial', include_barriers=False)


### 2.1 Constructor Validation

The constructor validates all parameters strictly with informative error messages.

In [8]:
# --- n_features validation ---
print("=== n_features validation ===")

# Must be at least 2
for bad_n in [1, 0, -1]:
    try:
        SymmetryInspiredFeatureMap(n_features=bad_n)
    except ValueError as e:
        print(f"  n_features={bad_n}: ValueError — {e}")

# Must be an integer, not float
try:
    SymmetryInspiredFeatureMap(n_features=4.0)
except TypeError as e:
    print(f"  n_features=4.0: TypeError — {e}")

# numpy integers are accepted
enc_np = SymmetryInspiredFeatureMap(n_features=np.int64(4))
print(f"  n_features=np.int64(4): OK — n_features={enc_np.n_features}")

=== n_features validation ===
  n_features=1: ValueError — n_features must be at least 2 for symmetry-inspired encoding, got 1
  n_features=0: ValueError — n_features must be at least 2 for symmetry-inspired encoding, got 0
  n_features=-1: ValueError — n_features must be at least 2 for symmetry-inspired encoding, got -1
  n_features=4.0: TypeError — n_features must be an integer, got float
  n_features=np.int64(4): OK — n_features=4


In [9]:
# --- symmetry validation ---
print("=== symmetry validation ===")

for bad_sym in ['invalid', 'rotational', 'SO2']:
    try:
        SymmetryInspiredFeatureMap(n_features=4, symmetry=bad_sym)
    except ValueError as e:
        print(f"  symmetry={bad_sym!r}: ValueError — {e}")

# Must be a string
try:
    SymmetryInspiredFeatureMap(n_features=4, symmetry=42)
except TypeError as e:
    print(f"  symmetry=42: TypeError — {e}")

=== symmetry validation ===
  symmetry='invalid': ValueError — symmetry must be one of ['cyclic', 'full', 'reflection', 'rotation'], got 'invalid'
  symmetry='rotational': ValueError — symmetry must be one of ['cyclic', 'full', 'reflection', 'rotation'], got 'rotational'
  symmetry='SO2': ValueError — symmetry must be one of ['cyclic', 'full', 'reflection', 'rotation'], got 'SO2'
  symmetry=42: TypeError — symmetry must be a string, got int


In [10]:
# --- rotation + odd n_features ---
print("=== rotation symmetry requires even n_features ===")

try:
    SymmetryInspiredFeatureMap(n_features=3, symmetry='rotation')
except ValueError as e:
    print(f"  n_features=3, symmetry='rotation': ValueError — {e}")

# Even numbers work fine
enc = SymmetryInspiredFeatureMap(n_features=6, symmetry='rotation')
print(f"  n_features=6, symmetry='rotation': OK ✓")

=== rotation symmetry requires even n_features ===
  n_features=3, symmetry='rotation': ValueError — rotation symmetry requires even n_features (for coordinate pairs), got 3
  n_features=6, symmetry='rotation': OK ✓


In [11]:
# --- reps validation ---
print("=== reps validation ===")

for bad_reps in [0, -1]:
    try:
        SymmetryInspiredFeatureMap(n_features=4, reps=bad_reps)
    except ValueError as e:
        print(f"  reps={bad_reps}: ValueError — {e}")

# Float raises TypeError
try:
    SymmetryInspiredFeatureMap(n_features=4, reps=1.5)
except TypeError as e:
    print(f"  reps=1.5: TypeError — {e}")

# Note: bool IS a subclass of int in Python, so True/False are accepted as ints
# True → reps=1 (valid), False → reps=0 (ValueError, not TypeError)
enc_true = SymmetryInspiredFeatureMap(n_features=4, reps=True)  # reps=1
print(f"  reps=True: accepted (reps={enc_true.reps}, because bool is int subclass)")

try:
    SymmetryInspiredFeatureMap(n_features=4, reps=False)  # reps=0 → ValueError
except ValueError as e:
    print(f"  reps=False: ValueError — {e} (because int(False)=0 < 1)")

=== reps validation ===
  reps=0: ValueError — reps must be at least 1, got 0
  reps=-1: ValueError — reps must be at least 1, got -1
  reps=1.5: TypeError — reps must be an integer, got float
  reps=True: accepted (reps=1, because bool is int subclass)
  reps=False: ValueError — reps must be at least 1, got 0 (because int(False)=0 < 1)


In [12]:
# --- entanglement validation ---
print("=== entanglement validation ===")

try:
    SymmetryInspiredFeatureMap(n_features=4, entanglement='all_to_all')
except ValueError as e:
    print(f"  entanglement='all_to_all': ValueError — {e}")

try:
    SymmetryInspiredFeatureMap(n_features=4, entanglement=True)
except TypeError as e:
    print(f"  entanglement=True: TypeError — {e}")

=== entanglement validation ===
  entanglement='all_to_all': ValueError — entanglement must be one of ['circular', 'full', 'linear', 'none'], got 'all_to_all'
  entanglement=True: TypeError — entanglement must be a string, got bool


In [13]:
# --- feature_map validation ---
print("=== feature_map validation ===")

try:
    SymmetryInspiredFeatureMap(n_features=4, feature_map='cosine')
except ValueError as e:
    print(f"  feature_map='cosine': ValueError — {e}")

try:
    SymmetryInspiredFeatureMap(n_features=4, feature_map=None)
except TypeError as e:
    print(f"  feature_map=None: TypeError — {e}")

=== feature_map validation ===
  feature_map='cosine': ValueError — feature_map must be one of ['angle', 'fourier', 'polynomial'], got 'cosine'
  feature_map=None: TypeError — feature_map must be a string, got NoneType


In [14]:
# --- include_barriers validation ---
print("=== include_barriers validation ===")

# Must be exactly bool, not int (even though bool is subclass of int)
try:
    SymmetryInspiredFeatureMap(n_features=4, include_barriers=1)
except TypeError as e:
    print(f"  include_barriers=1: TypeError — {e}")

=== include_barriers validation ===
  include_barriers=1: TypeError — include_barriers must be a bool, got int


### 2.2 Behavioral Warnings

The constructor emits warnings in specific edge-case configurations.

In [15]:
import warnings

# Warning 1: rotation symmetry overrides entanglement parameter
print("=== rotation + non-linear entanglement warning ===")
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', entanglement='full')
    if w:
        print(f"  UserWarning: {w[0].message}")
    # The encoding still works — rotation always uses coordinate pairs
    print(f"  Actual entanglement pairs: {enc.get_entanglement_pairs()}")

=== rotation + non-linear entanglement warning ===
  Actual entanglement pairs: [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


In [16]:
# Warning 2: full entanglement with many features (scalability)
print("=== full entanglement + many features warning ===")
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    enc_large = SymmetryInspiredFeatureMap(
        n_features=22, symmetry='cyclic', entanglement='full'
    )
    if w:
        for warning in w:
            print(f"  UserWarning: {warning.message}")
    n_pairs = len(enc_large.get_entanglement_pairs())
    print(f"  22 features × full entanglement = {n_pairs} pairs")

=== full entanglement + many features warning ===
  22 features × full entanglement = 231 pairs


---
## 3. Core Properties

SymmetryInspiredFeatureMap exposes several properties inherited from `BaseEncoding` plus its own attributes.

In [17]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)

print("=== Core Properties ===")
print(f"  n_features      : {enc.n_features}")
print(f"  n_qubits        : {enc.n_qubits}   (always == n_features)")
print(f"  depth           : {enc.depth}")
print(f"  symmetry        : {enc.symmetry!r}")
print(f"  reps            : {enc.reps}")
print(f"  entanglement    : {enc.entanglement!r}")
print(f"  feature_map     : {enc.feature_map!r}")
print(f"  include_barriers: {enc.include_barriers}")

assert enc.n_qubits == enc.n_features, "n_qubits always equals n_features"
print("\n  n_qubits == n_features ✓")

=== Core Properties ===
  n_features      : 4
  n_qubits        : 4   (always == n_features)
  depth           : 8
  symmetry        : 'rotation'
  reps            : 2
  entanglement    : 'linear'
  feature_map     : 'angle'
  include_barriers: True

  n_qubits == n_features ✓


In [18]:
# The config property returns a defensive copy of constructor kwargs
config = enc.config
print(f"config = {config}")

# Modifying the copy does NOT affect the encoding
config['reps'] = 999
assert enc.reps == 2, "Original encoding is unchanged"
print("Defensive copy: modifying config doesn't affect encoding ✓")

config = {'symmetry': 'rotation', 'reps': 2, 'entanglement': 'linear', 'feature_map': 'angle', 'include_barriers': True}
Defensive copy: modifying config doesn't affect encoding ✓


---
## 4. Encoding Properties (Lazy, Thread-Safe)

The `properties` attribute returns an `EncodingProperties` frozen dataclass with circuit metrics. It is lazily computed on first access and cached (thread-safe via double-checked locking).

In [19]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
props = enc.properties

print(f"type: {type(props).__name__}")
print(f"\n=== Required Fields ===")
print(f"  n_qubits          : {props.n_qubits}")
print(f"  depth              : {props.depth}")
print(f"  gate_count         : {props.gate_count}")
print(f"  single_qubit_gates : {props.single_qubit_gates}")
print(f"  two_qubit_gates    : {props.two_qubit_gates}")
print(f"  parameter_count    : {props.parameter_count}  (0 = all data-dependent, no trainable params)")
print(f"  is_entangling      : {props.is_entangling}")
print(f"  simulability       : {props.simulability!r}")
print(f"\n=== Optional Fields ===")
print(f"  trainability_estimate : {props.trainability_estimate}")
print(f"  notes                 : {props.notes!r}")

# Verify gate count consistency
assert props.single_qubit_gates + props.two_qubit_gates == props.gate_count
print("\n  single_qubit + two_qubit == gate_count ✓")

type: EncodingProperties

=== Required Fields ===
  n_qubits          : 4
  depth              : 8
  gate_count         : 40
  single_qubit_gates : 32
  two_qubit_gates    : 8
  parameter_count    : 0  (0 = all data-dependent, no trainable params)
  is_entangling      : True
  simulability       : 'not_simulable'

=== Optional Fields ===
  trainability_estimate : 0.55
  notes                 : 'Symmetry: rotation, Entanglement: linear, Feature map: angle, Coordinate pairs: 2'

  single_qubit + two_qubit == gate_count ✓


In [20]:
# Properties are frozen (immutable)
from dataclasses import FrozenInstanceError

try:
    props.n_qubits = 10
except FrozenInstanceError:
    print("FrozenInstanceError: properties are immutable ✓")

# Properties are cached — same object on repeated access
props_1 = enc.properties
props_2 = enc.properties
assert props_1 is props_2, "Same object returned"
print(f"Cached: id(props_1) == id(props_2) = {id(props_1) == id(props_2)} ✓")

FrozenInstanceError: properties are immutable ✓
Cached: id(props_1) == id(props_2) = True ✓


In [21]:
# Properties can be converted to a dictionary
props_dict = props.to_dict()
print("props.to_dict():")
for key, value in props_dict.items():
    print(f"  {key}: {value}")

props.to_dict():
  n_qubits: 4
  depth: 8
  gate_count: 40
  single_qubit_gates: 32
  two_qubit_gates: 8
  parameter_count: 0
  is_entangling: True
  simulability: not_simulable
  expressibility: None
  entanglement_capability: None
  trainability_estimate: 0.55
  noise_resilience_estimate: None
  notes: Symmetry: rotation, Entanglement: linear, Feature map: angle, Coordinate pairs: 2


---
## 5. Symmetry Types — In Depth

Each symmetry type uses different mathematical functions to compute gate angles, providing inductive bias for different kinds of data symmetries.

| Symmetry | Inspired by | Equivariant Angle | Interaction Angle | Best for |
|---|---|---|---|---|
| `rotation` | SO(2) | `√(x²+y²)` (radius) | `√(θᵢ²+θⱼ²)` (rotation-invariant) | Spatial/image data with rotational patterns |
| `cyclic` | Z_n | `xᵢ` (raw) | `(π-θᵢ)(π-θⱼ)` (symmetric coupling) | Time-series, periodic/sequential data |
| `reflection` | Z₂ | `(xᵢ+x_{n-1-i})/2` (mirror average) | `θᵢ·θⱼ` (product) | Palindromic/mirror-symmetric data |
| `full` | S₂ | `xᵢ + mean(x)` (symmetric polynomial) | `θᵢ·θⱼ` (product) | General permutation-like symmetries |

In [22]:
# get_symmetry_info() returns detailed symmetry configuration
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=6 if sym != 'rotation' else 6, symmetry=sym)
    info = enc.get_symmetry_info()
    print(f"=== {sym} symmetry ===")
    for key, value in info.items():
        print(f"  {key}: {value}")
    print()

=== rotation symmetry ===
  type: rotation
  entanglement_pattern: linear
  entanglement_pairs: [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
  n_entanglement_pairs: 5
  n_pairs: 3
  pair_indices: [(0, 1), (2, 3), (4, 5)]

=== cyclic symmetry ===
  type: cyclic
  entanglement_pattern: linear
  entanglement_pairs: [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
  n_entanglement_pairs: 5
  generator_shift: 1

=== reflection symmetry ===
  type: reflection
  entanglement_pattern: linear
  entanglement_pairs: [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
  n_entanglement_pairs: 5
  mirror_pairs: [(0, 5), (1, 4), (2, 3)]

=== full symmetry ===
  type: full
  entanglement_pattern: linear
  entanglement_pairs: [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
  n_entanglement_pairs: 5
  swap_pairs: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]



In [23]:
# Demonstrate how each symmetry computes angles differently
x = np.array([0.3, 0.7, 0.5, 0.2])

print("=== Encoding angles for x = [0.3, 0.7, 0.5, 0.2] ===\n")
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym)
    angles = enc.compute_encoding_angles(x)
    print(f"--- {sym} ---")
    print(f"  encoding    : {angles['encoding']}")
    print(f"  equivariant : {angles['equivariant']}")
    print(f"  interaction : {angles['interaction']}")
    print()

=== Encoding angles for x = [0.3, 0.7, 0.5, 0.2] ===

--- rotation ---
  encoding    : [0.3 0.7 0.5 0.2]
  equivariant : [0.76157731 0.76157731 0.53851648 0.53851648]
  interaction : [0.76157731 0.86023253 0.53851648]

--- cyclic ---
  encoding    : [0.3 0.7 0.5 0.2]
  equivariant : [0.3 0.7 0.5 0.2]
  interaction : [6.93801175 6.44969322 7.77048954]

--- reflection ---
  encoding    : [0.3 0.7 0.5 0.2]
  equivariant : [0.25 0.6  0.6  0.25]
  interaction : [0.21 0.35 0.1 ]

--- full ---
  encoding    : [0.3 0.7 0.5 0.2]
  equivariant : [0.725 1.125 0.925 0.625]
  interaction : [0.21 0.35 0.1 ]



In [24]:
# Verify rotation symmetry uses radius: sqrt(x^2 + y^2)
enc_rot = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation')
x = np.array([3.0, 4.0, 1.0, 0.0])  # radius should be 5.0 and 1.0
angles = enc_rot.compute_encoding_angles(x)

# Both qubits in a pair share the same equivariant angle (the radius)
assert np.isclose(angles['equivariant'][0], 5.0)  # sqrt(3^2 + 4^2) = 5
assert np.isclose(angles['equivariant'][1], 5.0)  # same pair
assert np.isclose(angles['equivariant'][2], 1.0)  # sqrt(1^2 + 0^2) = 1
assert np.isclose(angles['equivariant'][3], 1.0)  # same pair
print("Rotation equivariant angles = radii of coordinate pairs ✓")
print(f"  Pair (0,1): radius = sqrt(3² + 4²) = {angles['equivariant'][0]:.1f}")
print(f"  Pair (2,3): radius = sqrt(1² + 0²) = {angles['equivariant'][2]:.1f}")

Rotation equivariant angles = radii of coordinate pairs ✓
  Pair (0,1): radius = sqrt(3² + 4²) = 5.0
  Pair (2,3): radius = sqrt(1² + 0²) = 1.0


In [25]:
# Verify reflection symmetry uses mirror average: (x[i] + x[n-1-i]) / 2
enc_ref = SymmetryInspiredFeatureMap(n_features=4, symmetry='reflection')
x = np.array([0.2, 0.8, 0.6, 1.0])
# Mirror pairs: (0, 3) and (1, 2)
angles = enc_ref.compute_encoding_angles(x)

assert np.isclose(angles['equivariant'][0], (0.2 + 1.0) / 2)  # = 0.6
assert np.isclose(angles['equivariant'][3], (1.0 + 0.2) / 2)  # = 0.6 (same!)
assert np.isclose(angles['equivariant'][1], (0.8 + 0.6) / 2)  # = 0.7
assert np.isclose(angles['equivariant'][2], (0.6 + 0.8) / 2)  # = 0.7 (same!)
print("Reflection equivariant angles = mirror averages ✓")
print(f"  Mirror (0,3): ({x[0]} + {x[3]}) / 2 = {angles['equivariant'][0]}")
print(f"  Mirror (1,2): ({x[1]} + {x[2]}) / 2 = {angles['equivariant'][1]}")

Reflection equivariant angles = mirror averages ✓
  Mirror (0,3): (0.2 + 1.0) / 2 = 0.6
  Mirror (1,2): (0.8 + 0.6) / 2 = 0.7


In [26]:
# Verify full symmetry uses x[i] + mean(x)
enc_full = SymmetryInspiredFeatureMap(n_features=4, symmetry='full')
x = np.array([1.0, 2.0, 3.0, 4.0])
mean_x = np.mean(x)  # 2.5
angles = enc_full.compute_encoding_angles(x)

for i in range(4):
    expected = x[i] + mean_x
    assert np.isclose(angles['equivariant'][i], expected)
print(f"Full equivariant angles = x[i] + mean(x) ✓")
print(f"  mean(x) = {mean_x}")
for i in range(4):
    print(f"  qubit {i}: {x[i]} + {mean_x} = {angles['equivariant'][i]}")

Full equivariant angles = x[i] + mean(x) ✓
  mean(x) = 2.5
  qubit 0: 1.0 + 2.5 = 3.5
  qubit 1: 2.0 + 2.5 = 4.5
  qubit 2: 3.0 + 2.5 = 5.5
  qubit 3: 4.0 + 2.5 = 6.5


---
## 6. Feature Maps — angle, fourier, polynomial

The `feature_map` parameter controls how raw feature values are mapped to rotation angles in the encoding layer.

| Feature Map | Formula | Recommended Input Range | Use Case |
|---|---|---|---|
| `angle` | θ = x | [0, π] | General purpose, pre-scaled data |
| `fourier` | θ = 2πx | [0, 1] | Periodic/Fourier-based data |
| `polynomial` | θ = x + x² | [-1, 1] | Non-linear feature interactions |

In [27]:
# Verify feature map formulas using compute_encoding_angles
x = np.array([0.5, 1.0, 0.25, 0.75])

for fm in ['angle', 'fourier', 'polynomial']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', feature_map=fm)
    angles = enc.compute_encoding_angles(x)
    print(f"--- feature_map='{fm}' ---")
    print(f"  encoding angles: {angles['encoding']}")

# Manual verification
print("\n=== Manual verification ===")
print(f"  angle:      x = 0.5 → θ = {0.5}")
print(f"  fourier:    x = 0.5 → θ = 2π × 0.5 = {2 * np.pi * 0.5:.4f}")
print(f"  polynomial: x = 0.5 → θ = 0.5 + 0.5² = {0.5 + 0.5**2}")

--- feature_map='angle' ---
  encoding angles: [0.5  1.   0.25 0.75]
--- feature_map='fourier' ---
  encoding angles: [3.14159265 6.28318531 1.57079633 4.71238898]
--- feature_map='polynomial' ---
  encoding angles: [0.75   2.     0.3125 1.3125]

=== Manual verification ===
  angle:      x = 0.5 → θ = 0.5
  fourier:    x = 0.5 → θ = 2π × 0.5 = 3.1416
  polynomial: x = 0.5 → θ = 0.5 + 0.5² = 0.75


In [28]:
# Input range warnings — each feature map has a recommended range
import warnings

for fm, good_range, bad_val in [
    ('angle', '[0, π]', np.array([5.0, 5.0, 5.0, 5.0])),       # > π
    ('fourier', '[0, 1]', np.array([2.0, 2.0, 2.0, 2.0])),      # > 1
    ('polynomial', '[-1, 1]', np.array([3.0, 3.0, 3.0, 3.0])),  # > 1
]:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', feature_map=fm)
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        _ = enc.get_circuit(bad_val, backend='pennylane')
        warned = any("range" in str(warning.message).lower() or "out" in str(warning.message).lower()
                      for warning in w)
        print(f"  feature_map='{fm}', recommended={good_range}, input={bad_val[0]}: "
              f"{'Warning emitted' if warned else 'No warning'}")

  feature_map='angle', recommended=[0, π], input=5.0: Warning emitted
  feature_map='fourier', recommended=[0, 1], input=2.0: Warning emitted
  feature_map='polynomial', recommended=[-1, 1], input=3.0: Warning emitted


---
## 7. Entanglement Patterns

The `entanglement` parameter controls how qubits are connected via two-qubit gates.

| Pattern | Pairs (n=4) | Pairs (n=5) | Depth Contribution |
|---|---|---|---|
| `none` | [] | [] | 0 |
| `linear` | [(0,1), (1,2), (2,3)] | [(0,1), (1,2), (2,3), (3,4)] | 1 (n≤2) or 2 |
| `circular` | [(0,1), (1,2), (2,3), (3,0)] | [(0,1), (1,2), (2,3), (3,4), (4,0)] | 2 (even) or 3 (odd) |
| `full` | [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)] | all 10 pairs | n-1 (even) or n (odd) |

In [29]:
# get_entanglement_pairs() returns the qubit connections
for ent in ['none', 'linear', 'circular', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=5, symmetry='cyclic', entanglement=ent)
    pairs = enc.get_entanglement_pairs()
    print(f"entanglement={ent!r:12s}  pairs={pairs}")
    # Verify it returns a copy (defensive)
    pairs_copy = enc.get_entanglement_pairs()
    assert pairs == pairs_copy
    assert pairs is not pairs_copy, "Returns a copy, not the internal list"

print("\nDefensive copy: get_entanglement_pairs() returns a new list each time ✓")

entanglement='none'        pairs=[]
entanglement='linear'      pairs=[(0, 1), (1, 2), (2, 3), (3, 4)]
entanglement='circular'    pairs=[(0, 1), (1, 2), (2, 3), (3, 4), (4, 0)]
entanglement='full'        pairs=[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

Defensive copy: get_entanglement_pairs() returns a new list each time ✓


In [30]:
# Entanglement affects circuit depth
print("=== Depth by entanglement pattern (n=6, cyclic symmetry, reps=1) ===")
for ent in ['none', 'linear', 'circular', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=6, symmetry='cyclic', reps=1, entanglement=ent)
    pairs = enc.get_entanglement_pairs()
    print(f"  {ent:10s}: depth={enc.depth:3d}, n_pairs={len(pairs):3d}")

=== Depth by entanglement pattern (n=6, cyclic symmetry, reps=1) ===
  none      : depth=  3, n_pairs=  0
  linear    : depth=  5, n_pairs=  5
  circular  : depth=  5, n_pairs=  6
  full      : depth=  8, n_pairs= 15


In [31]:
# Rotation symmetry: entanglement PAIRS follow the entanglement pattern,
# but the GATE TYPE used is always CRZ (rotation-invariant) and the
# entanglement DEPTH assumes coordinate-pair disjointness.
# The warning says "overridden" because the interaction structure changes.
import warnings

print("=== Rotation symmetry entanglement pairs ===")
for ent in ['none', 'linear', 'circular', 'full']:
    with warnings.catch_warnings(record=True):
        warnings.simplefilter("always")
        enc = SymmetryInspiredFeatureMap(
            n_features=6, symmetry='rotation', entanglement=ent
        )
    pairs = enc.get_entanglement_pairs()
    print(f"  entanglement={ent!r:12s}  n_pairs={len(pairs):2d}  pairs={pairs}")

=== Rotation symmetry entanglement pairs ===
  entanglement='none'        n_pairs= 0  pairs=[]
  entanglement='linear'      n_pairs= 5  pairs=[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]
  entanglement='circular'    n_pairs= 6  pairs=[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
  entanglement='full'        n_pairs=15  pairs=[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]


In [32]:
# Entanglement determines whether the state is entangled
enc_none = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', entanglement='none')
enc_linear = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', entanglement='linear')

assert enc_none.properties.is_entangling == False
assert enc_linear.properties.is_entangling == True
print(f"  entanglement='none':   is_entangling = {enc_none.properties.is_entangling}")
print(f"  entanglement='linear': is_entangling = {enc_linear.properties.is_entangling}")

# This also affects simulability
assert enc_none.properties.simulability == "simulable"
assert enc_linear.properties.simulability == "not_simulable"
print(f"  entanglement='none':   simulability  = {enc_none.properties.simulability!r}")
print(f"  entanglement='linear': simulability  = {enc_linear.properties.simulability!r}")

  entanglement='none':   is_entangling = False
  entanglement='linear': is_entangling = True
  entanglement='none':   simulability  = 'simulable'
  entanglement='linear': simulability  = 'not_simulable'


---
## 8. Circuit Generation — PennyLane Backend

PennyLane is the default backend. `get_circuit()` returns a **callable** (closure) that applies quantum gates when called inside a QNode.

In [33]:
import pennylane as qml

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=1)
x = np.array([0.5, 1.0, 1.5, 2.0])

# get_circuit returns a callable
circuit_fn = enc.get_circuit(x, backend='pennylane')
print(f"Type: {type(circuit_fn).__name__}")
print(f"Callable: {callable(circuit_fn)}")

Type: function
Callable: True


In [34]:
# Use inside a QNode to get a statevector
dev = qml.device("default.qubit", wires=enc.n_qubits)

@qml.qnode(dev)
def get_state(x_input):
    circuit_fn = enc.get_circuit(x_input, backend='pennylane')
    circuit_fn()
    return qml.state()

state = get_state(x)
print(f"Statevector shape: {state.shape}")
print(f"Statevector norm:  {np.linalg.norm(state):.10f}")
assert np.isclose(np.linalg.norm(state), 1.0)
print("State is normalized ✓")

Statevector shape: (16,)
Statevector norm:  1.0000000000
State is normalized ✓


In [35]:
# Visualize the circuit using PennyLane's drawer
@qml.qnode(dev)
def draw_circuit(x_input):
    circuit_fn = enc.get_circuit(x_input, backend='pennylane')
    circuit_fn()
    return qml.state()

print(qml.draw(draw_circuit)(x))

0: ──H──RY(0.50)──RZ(1.12)─╭●────────┤  State
1: ──H──RY(1.00)──RZ(1.12)─╰RZ(1.12)─┤  State
2: ──H──RY(1.50)──RZ(2.50)─╭●────────┤  State
3: ──H──RY(2.00)──RZ(2.50)─╰RZ(2.50)─┤  State


In [36]:
# With more reps, the circuit goes deeper
enc_3reps = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=3)
dev3 = qml.device("default.qubit", wires=4)

@qml.qnode(dev3)
def draw_3reps(x_input):
    fn = enc_3reps.get_circuit(x_input, backend='pennylane')
    fn()
    return qml.state()

print(f"reps=3, depth={enc_3reps.depth}")
print(qml.draw(draw_3reps)(x))

reps=3, depth=12
0: ──H──RY(0.50)──RZ(1.12)─╭●─────────H──RY(0.50)──RZ(1.12)─╭●─────────H──RY(0.50)──RZ(1.12) ···
1: ──H──RY(1.00)──RZ(1.12)─╰RZ(1.12)──H──RY(1.00)──RZ(1.12)─╰RZ(1.12)──H──RY(1.00)──RZ(1.12) ···
2: ──H──RY(1.50)──RZ(2.50)─╭●─────────H──RY(1.50)──RZ(2.50)─╭●─────────H──RY(1.50)──RZ(2.50) ···
3: ──H──RY(2.00)──RZ(2.50)─╰RZ(2.50)──H──RY(2.00)──RZ(2.50)─╰RZ(2.50)──H──RY(2.00)──RZ(2.50) ···

0: ··· ─╭●────────┤  State
1: ··· ─╰RZ(1.12)─┤  State
2: ··· ─╭●────────┤  State
3: ··· ─╰RZ(2.50)─┤  State


---
## 9. Circuit Generation — Qiskit Backend

The Qiskit backend returns a `QuantumCircuit` object with named gates and optional barriers.

In [37]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=1)
x = np.array([0.5, 1.0, 1.5, 2.0])

qc = enc.get_circuit(x, backend='qiskit')
print(f"Type: {type(qc).__name__}")
print(f"Name: {qc.name}")
print(f"Qubits: {qc.num_qubits}")
print(f"Circuit depth: {qc.depth()}")
print(f"\nCircuit diagram:")
print(qc.draw(output='text'))

Type: QuantumCircuit
Name: SymmetryInspiredFeatureMap
Qubits: 4
Circuit depth: 4

Circuit diagram:
     ┌───┐┌─────────┐ ░ ┌───────────┐ ░              
q_0: ┤ H ├┤ Ry(0.5) ├─░─┤ Rz(1.118) ├─░───────■──────
     ├───┤└┬───────┬┘ ░ ├───────────┤ ░ ┌─────┴─────┐
q_1: ┤ H ├─┤ Ry(1) ├──░─┤ Rz(1.118) ├─░─┤ Rz(1.118) ├
     ├───┤┌┴───────┴┐ ░ └┬─────────┬┘ ░ └───────────┘
q_2: ┤ H ├┤ Ry(1.5) ├─░──┤ Rz(2.5) ├──░───────■──────
     ├───┤└┬───────┬┘ ░  ├─────────┤  ░  ┌────┴────┐ 
q_3: ┤ H ├─┤ Ry(2) ├──░──┤ Rz(2.5) ├──░──┤ Rz(2.5) ├─
     └───┘ └───────┘  ░  └─────────┘  ░  └─────────┘ 


In [38]:
# Barriers between layers (default: include_barriers=True)
enc_barriers = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', reps=1, include_barriers=True)
enc_no_barriers = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', reps=1, include_barriers=False)

qc_b = enc_barriers.get_circuit(x, backend='qiskit')
qc_nb = enc_no_barriers.get_circuit(x, backend='qiskit')

print("=== With barriers ===")
print(qc_b.draw(output='text'))
print("\n=== Without barriers ===")
print(qc_nb.draw(output='text'))

=== With barriers ===
     ┌───┐┌─────────┐ ░ ┌─────────┐ ░                              »
q_0: ┤ H ├┤ Ry(0.5) ├─░─┤ Rz(0.5) ├─░───■──────────────────■───────»
     ├───┤└┬───────┬┘ ░ └┬───────┬┘ ░ ┌─┴─┐┌────────────┐┌─┴─┐     »
q_1: ┤ H ├─┤ Ry(1) ├──░──┤ Rz(1) ├──░─┤ X ├┤ Rz(5.6572) ├┤ X ├──■──»
     ├───┤┌┴───────┴┐ ░ ┌┴───────┴┐ ░ └───┘└────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ Ry(1.5) ├─░─┤ Rz(1.5) ├─░─────────────────────────┤ X ├»
     ├───┤└┬───────┬┘ ░ └┬───────┬┘ ░                         └───┘»
q_3: ┤ H ├─┤ Ry(2) ├──░──┤ Rz(2) ├──░──────────────────────────────»
     └───┘ └───────┘  ░  └───────┘  ░                              »
«                                               
«q_0: ──────────────────────────────────────────
«                                               
«q_1: ────────────────■─────────────────────────
«     ┌────────────┐┌─┴─┐                       
«q_2: ┤ Rz(3.5156) ├┤ X ├──■─────────────────■──
«     └────────────┘└───┘┌─┴─┐┌───────────┐┌─┴─┐
«q_3: ────────

In [39]:
# All symmetry types with Qiskit
print("=== All symmetry types (Qiskit, 4 qubits, reps=1) ===\n")
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym, reps=1)
    qc = enc.get_circuit(x, backend='qiskit')
    print(f"--- {sym} (depth={qc.depth()}) ---")
    print(qc.draw(output='text'))
    print()

=== All symmetry types (Qiskit, 4 qubits, reps=1) ===

--- rotation (depth=4) ---
     ┌───┐┌─────────┐ ░ ┌───────────┐ ░              
q_0: ┤ H ├┤ Ry(0.5) ├─░─┤ Rz(1.118) ├─░───────■──────
     ├───┤└┬───────┬┘ ░ ├───────────┤ ░ ┌─────┴─────┐
q_1: ┤ H ├─┤ Ry(1) ├──░─┤ Rz(1.118) ├─░─┤ Rz(1.118) ├
     ├───┤┌┴───────┴┐ ░ └┬─────────┬┘ ░ └───────────┘
q_2: ┤ H ├┤ Ry(1.5) ├─░──┤ Rz(2.5) ├──░───────■──────
     ├───┤└┬───────┬┘ ░  ├─────────┤  ░  ┌────┴────┐ 
q_3: ┤ H ├─┤ Ry(2) ├──░──┤ Rz(2.5) ├──░──┤ Rz(2.5) ├─
     └───┘ └───────┘  ░  └─────────┘  ░  └─────────┘ 

--- cyclic (depth=12) ---
     ┌───┐┌─────────┐ ░ ┌─────────┐ ░                              »
q_0: ┤ H ├┤ Ry(0.5) ├─░─┤ Rz(0.5) ├─░───■──────────────────■───────»
     ├───┤└┬───────┬┘ ░ └┬───────┬┘ ░ ┌─┴─┐┌────────────┐┌─┴─┐     »
q_1: ┤ H ├─┤ Ry(1) ├──░──┤ Rz(1) ├──░─┤ X ├┤ Rz(5.6572) ├┤ X ├──■──»
     ├───┤┌┴───────┴┐ ░ ┌┴───────┴┐ ░ └───┘└────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ Ry(1.5) ├─░─┤ Rz(1.5) ├─░───────────────────────

---
## 10. Circuit Generation — Cirq Backend

The Cirq backend returns a `cirq.Circuit` with optimized `Moment` placement (non-overlapping operations are batched together).

In [40]:
import cirq

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=1)
x = np.array([0.5, 1.0, 1.5, 2.0])

cirq_circuit = enc.get_circuit(x, backend='cirq')
print(f"Type: {type(cirq_circuit).__name__}")
print(f"Moments: {len(cirq_circuit)}")
print(f"\n{cirq_circuit}")

Type: Circuit
Moments: 7

0: ───H───Ry(0.159π)───Rz(0.356π)────────────────@─────────────────@───
                                                 │                 │
1: ───H───Ry(0.318π)───Rz(0.356π)───Rz(0.178π)───X───Rz(-0.178π)───X───

2: ───H───Ry(0.477π)───Rz(0.796π)────────────────@─────────────────@───
                                                 │                 │
3: ───H───Ry(0.637π)───Rz(0.796π)───Rz(0.398π)───X───Rz(-0.398π)───X───


In [41]:
# Simulate to get statevector
simulator = cirq.Simulator()
result = simulator.simulate(cirq_circuit)
state = result.final_state_vector
print(f"Statevector shape: {state.shape}")
print(f"Norm: {np.linalg.norm(state):.10f}")
assert np.isclose(np.linalg.norm(state), 1.0)
print("Normalized ✓")

Statevector shape: (16,)
Norm: 1.0000000000
Normalized ✓


---
## 11. Cross-Backend State Equivalence

All three backends produce equivalent quantum states. However, they use different qubit ordering conventions:
- **PennyLane**: Little-endian (qubit 0 is least significant bit)
- **Qiskit**: Big-endian (qubit 0 is most significant bit)
- **Cirq**: Big-endian (like Qiskit)

The `convert_state_vector_ordering()` utility handles conversion.

In [42]:
from encoding_atlas.encodings.symmetry_inspired_feature_map import convert_state_vector_ordering
from encoding_atlas.analysis import simulate_encoding_statevector

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=1)
x = np.array([0.5, 1.0, 1.5, 2.0])

# Get states from all backends via the analysis utility
# (simulate_encoding_statevector returns MSB convention for all backends)
state_pl = simulate_encoding_statevector(enc, x, backend='pennylane')
state_qk = simulate_encoding_statevector(enc, x, backend='qiskit')
state_cq = simulate_encoding_statevector(enc, x, backend='cirq')

# The analysis utility normalizes ordering, so probability distributions match
probs_pl = np.abs(state_pl) ** 2
probs_qk = np.abs(state_qk) ** 2
probs_cq = np.abs(state_cq) ** 2

print("Probability distributions match across backends:")
print(f"  PennyLane ↔ Qiskit: {np.allclose(probs_pl, probs_qk, atol=1e-6)}")
print(f"  PennyLane ↔ Cirq:   {np.allclose(probs_pl, probs_cq, atol=1e-6)}")
print(f"  Qiskit ↔ Cirq:      {np.allclose(probs_qk, probs_cq, atol=1e-6)}")

Probability distributions match across backends:
  PennyLane ↔ Qiskit: True
  PennyLane ↔ Cirq:   True
  Qiskit ↔ Cirq:      True


In [43]:
# The convert_state_vector_ordering utility
n_qubits = 3
state = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])  # 2^3 = 8 amplitudes
state = state / np.linalg.norm(state)

# Convert big-endian to little-endian and back
converted = convert_state_vector_ordering(state, n_qubits, source='big', target='little')
restored = convert_state_vector_ordering(converted, n_qubits, source='little', target='big')

print(f"Original:  {np.round(state, 4)}")
print(f"Converted: {np.round(converted, 4)}")
print(f"Restored:  {np.round(restored, 4)}")
assert np.allclose(state, restored)
print("Round-trip conversion: big → little → big ✓")

# Same convention returns a copy
same = convert_state_vector_ordering(state, n_qubits, source='big', target='big')
assert np.allclose(state, same)
assert same is not state
print("Same-convention returns a copy ✓")

Original:  [0.07   0.14   0.21   0.2801 0.3501 0.4201 0.4901 0.5601]
Converted: [0.07   0.3501 0.21   0.4901 0.14   0.4201 0.2801 0.5601]
Restored:  [0.07   0.14   0.21   0.2801 0.3501 0.4201 0.4901 0.5601]
Round-trip conversion: big → little → big ✓
Same-convention returns a copy ✓


---
## 12. Batch Circuit Generation

`get_circuits()` generates circuits for multiple data samples at once, with optional parallel processing via `ThreadPoolExecutor`.

In [44]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
np.random.seed(42)
X_batch = np.random.uniform(0, np.pi, (10, 4))

# Sequential (default)
circuits_seq = enc.get_circuits(X_batch, backend='pennylane')
print(f"Sequential: {len(circuits_seq)} circuits, all callable: {all(callable(c) for c in circuits_seq)}")

# Parallel
circuits_par = enc.get_circuits(X_batch, backend='pennylane', parallel=True)
print(f"Parallel:   {len(circuits_par)} circuits, all callable: {all(callable(c) for c in circuits_par)}")

Sequential: 10 circuits, all callable: True
Parallel:   10 circuits, all callable: True


In [45]:
# Verify parallel and sequential produce identical states
dev = qml.device("default.qubit", wires=enc.n_qubits)

@qml.qnode(dev)
def run_circuit(fn):
    fn()
    return qml.state()

for i in range(len(X_batch)):
    state_seq = run_circuit(circuits_seq[i])
    state_par = run_circuit(circuits_par[i])
    assert np.allclose(state_seq, state_par, atol=1e-10)

print(f"All {len(X_batch)} states match between sequential and parallel ✓")

All 10 states match between sequential and parallel ✓


In [46]:
# Custom max_workers
import os
circuits_custom = enc.get_circuits(
    X_batch, backend='qiskit', parallel=True, max_workers=2
)
print(f"Custom max_workers=2: {len(circuits_custom)} Qiskit circuits generated ✓")

Custom max_workers=2: 10 Qiskit circuits generated ✓


In [47]:
# 1D input is automatically reshaped to 2D (single sample)
x_1d = np.array([0.1, 0.2, 0.3, 0.4])
circuits_1d = enc.get_circuits(x_1d, backend='pennylane')
print(f"1D input → {len(circuits_1d)} circuit(s) ✓")

1D input → 1 circuit(s) ✓


In [48]:
# Order is preserved with parallel processing
enc_order = SymmetryInspiredFeatureMap(n_features=2, symmetry='reflection')
dev_order = qml.device("default.qubit", wires=2)

X_distinct = np.array([
    [0.0, 0.0],
    [np.pi/2, 0.0],
    [0.0, np.pi/2],
    [np.pi, np.pi],
])

circuits_par = enc_order.get_circuits(X_distinct, backend='pennylane', parallel=True)

@qml.qnode(dev_order)
def get_state_from_fn(fn):
    fn()
    return qml.state()

print("Order verification:")
for i, fn in enumerate(circuits_par):
    state = get_state_from_fn(fn)
    # Also generate the expected state from get_circuit (single)
    fn_single = enc_order.get_circuit(X_distinct[i], backend='pennylane')
    @qml.qnode(dev_order)
    def get_expected():
        fn_single()
        return qml.state()
    expected = get_expected()
    match = np.allclose(state, expected, atol=1e-10)
    print(f"  Sample {i}: parallel state matches single-circuit state: {match}")
    assert match

print("Order preserved ✓")

Order verification:
  Sample 0: parallel state matches single-circuit state: True
  Sample 1: parallel state matches single-circuit state: True
  Sample 2: parallel state matches single-circuit state: True
  Sample 3: parallel state matches single-circuit state: True
Order preserved ✓


---
## 13. Input Validation & Edge Cases

`get_circuit()` thoroughly validates inputs before circuit construction.

In [49]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation')

# --- Shape validation ---
print("=== Shape validation ===")

# Wrong number of features
try:
    enc.get_circuit(np.array([0.1, 0.2, 0.3]))
except ValueError as e:
    print(f"  Wrong n_features: ValueError ✓")

# 3D input rejected
try:
    enc.get_circuit(np.ones((2, 2, 4)))
except ValueError as e:
    print(f"  3D input: ValueError ✓")

# 2D single-sample is accepted (auto-reshaped)
circuit_2d = enc.get_circuit(np.array([[0.1, 0.2, 0.3, 0.4]]), backend='pennylane')
print(f"  2D single-sample: accepted ✓")

=== Shape validation ===
  Wrong n_features: ValueError ✓
  3D input: ValueError ✓
  2D single-sample: accepted ✓


In [50]:
# --- Value validation ---
print("=== Value validation ===")

# NaN values
try:
    enc.get_circuit(np.array([0.1, np.nan, 0.3, 0.4]))
except ValueError as e:
    print(f"  NaN: ValueError ✓")

# Inf values
try:
    enc.get_circuit(np.array([0.1, np.inf, 0.3, 0.4]))
except ValueError as e:
    print(f"  Inf: ValueError ✓")

# Negative infinity
try:
    enc.get_circuit(np.array([0.1, -np.inf, 0.3, 0.4]))
except ValueError as e:
    print(f"  -Inf: ValueError ✓")

=== Value validation ===
  NaN: ValueError ✓
  Inf: ValueError ✓
  -Inf: ValueError ✓


In [51]:
# --- Type validation ---
print("=== Type validation ===")

# Complex numbers are rejected
try:
    enc.get_circuit(np.array([0.1+0j, 0.2+0j, 0.3+0j, 0.4+0j]))
except (TypeError, ValueError) as e:
    print(f"  Complex: {type(e).__name__} ✓")

# Strings are rejected
try:
    enc.get_circuit("invalid")
except (TypeError, ValueError) as e:
    print(f"  String: {type(e).__name__} ✓")

=== Type validation ===
  Complex: TypeError ✓
  String: ValueError ✓


In [52]:
# --- Accepted input formats ---
print("=== Accepted input formats ===")

# Python list → auto-converted to numpy array
c1 = enc.get_circuit([0.1, 0.2, 0.3, 0.4], backend='pennylane')
print(f"  Python list: accepted ✓")

# Tuple
c2 = enc.get_circuit((0.1, 0.2, 0.3, 0.4), backend='pennylane')
print(f"  Tuple: accepted ✓")

# Integer array
c3 = enc.get_circuit(np.array([1, 2, 3, 4]), backend='pennylane')
print(f"  Integer array: accepted ✓")

=== Accepted input formats ===
  Python list: accepted ✓
  Tuple: accepted ✓
  Integer array: accepted ✓


c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\IPython\core\interactiveshell.py:3579: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=1, max=4). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
# --- Backend validation ---
print("=== Backend validation ===")

try:
    enc.get_circuit(np.array([0.1, 0.2, 0.3, 0.4]), backend='tensorflow')
except ValueError as e:
    print(f"  backend='tensorflow': ValueError ✓")

=== Backend validation ===
  backend='tensorflow': ValueError ✓


In [54]:
# --- Defensive copy demonstration ---
print("=== Defensive copy (input isolation) ===")

x_mutable = np.array([0.1, 0.2, 0.3, 0.4])
_ = enc.get_circuit(x_mutable, backend='pennylane')

# Original array is not modified
x_mutable[0] = 999.0
print(f"  Original array modified to {x_mutable[0]}, encoding is unaffected ✓")

=== Defensive copy (input isolation) ===
  Original array modified to 999.0, encoding is unaffected ✓


---
## 14. Layer Structure

`get_layer_structure()` returns a detailed breakdown of the circuit architecture per repetition.

In [55]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2, entanglement='linear')
structure = enc.get_layer_structure()

print(f"n_qubits:     {structure['n_qubits']}")
print(f"n_reps:       {structure['n_reps']}")
print(f"total_layers: {structure['total_layers']}")
print(f"layers/rep:   {structure['layers_per_rep']}")
print(f"symmetry:     {structure['symmetry']}")
print(f"entanglement: {structure['entanglement']}")
print(f"feature_map:  {structure['feature_map']}")
print(f"\n=== Layer Details ===")
for layer in structure['layers']:
    print(f"  [{layer['index']}] Rep {layer['rep']}: {layer['name']}")
    print(f"      gate_types: {layer['gate_types']}")
    print(f"      n_gates:    {layer['n_gates']}")
    print(f"      {layer['description']}")
    print()

n_qubits:     4
n_reps:       2
total_layers: 8
layers/rep:   4
symmetry:     rotation
entanglement: linear
feature_map:  angle

=== Layer Details ===
  [0] Rep 0: hadamard
      gate_types: ['H']
      n_gates:    4
      Create superposition (rep 1)

  [1] Rep 0: encoding
      gate_types: ['RY']
      n_gates:    4
      Feature encoding via RY rotations (rep 1)

  [2] Rep 0: equivariant
      gate_types: ['RZ']
      n_gates:    4
      Symmetry-aware RZ rotations (rep 1)

  [3] Rep 0: entanglement
      gate_types: ['CRZ']
      n_gates:    3
      rotation-inspired entanglement (rep 1)

  [4] Rep 1: hadamard
      gate_types: ['H']
      n_gates:    4
      Create superposition (rep 2)

  [5] Rep 1: encoding
      gate_types: ['RY']
      n_gates:    4
      Feature encoding via RY rotations (rep 2)

  [6] Rep 1: equivariant
      gate_types: ['RZ']
      n_gates:    4
      Symmetry-aware RZ rotations (rep 2)

  [7] Rep 1: entanglement
      gate_types: ['CRZ']
      n_gates:   

In [56]:
# Compare layer structure across symmetry types
print("=== Layers per rep by symmetry ===")
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym, reps=1)
    s = enc.get_layer_structure()
    print(f"  {sym:12s}: {s['layers_per_rep']} layers, total depth={enc.depth}")

# Without entanglement: 3 layers per rep
enc_none = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', reps=1, entanglement='none')
s_none = enc_none.get_layer_structure()
print(f"\n  none-entanglement: {s_none['layers_per_rep']} layers per rep")
assert s_none['layers_per_rep'] == 3

=== Layers per rep by symmetry ===
  rotation    : 4 layers, total depth=4
  cyclic      : 4 layers, total depth=5
  reflection  : 4 layers, total depth=5
  full        : 4 layers, total depth=5

  none-entanglement: 3 layers per rep


---
## 15. Gate Count Breakdown

`gate_count_breakdown()` returns a `SymmetryInspiredGateBreakdown` TypedDict with exact gate counts by type.

In [57]:
from encoding_atlas.encodings.symmetry_inspired_feature_map import SymmetryInspiredGateBreakdown

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
breakdown = enc.gate_count_breakdown()

print(f"Type: {type(breakdown).__name__}")
print(f"\n=== Gate Counts ===")
for key, value in breakdown.items():
    print(f"  {key:24s}: {value}")

# Verify totals
assert breakdown['total_single_qubit'] + breakdown['total_two_qubit'] == breakdown['total']
print(f"\n  single + two-qubit == total ✓")

Type: dict

=== Gate Counts ===
  hadamard                : 8
  ry_encoding             : 8
  rz_equivariant          : 8
  rz_entanglement         : 8
  ry_entanglement         : 0
  cnot                    : 0
  crz                     : 4
  cz                      : 0
  total_single_qubit      : 32
  total_two_qubit         : 8
  total                   : 40

  single + two-qubit == total ✓


In [58]:
# Compare gate counts across all four symmetry types
print("=== Gate count comparison (n=4, reps=2) ===\n")
print(f"{'Gate Type':<20s} {'rotation':>10s} {'cyclic':>10s} {'reflection':>10s} {'full':>10s}")
print("-" * 62)

breakdowns = {}
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym, reps=2)
    breakdowns[sym] = enc.gate_count_breakdown()

for key in breakdowns['rotation'].keys():
    vals = [str(breakdowns[sym][key]) for sym in ['rotation', 'cyclic', 'reflection', 'full']]
    print(f"{key:<20s} {vals[0]:>10s} {vals[1]:>10s} {vals[2]:>10s} {vals[3]:>10s}")

=== Gate count comparison (n=4, reps=2) ===

Gate Type              rotation     cyclic reflection       full
--------------------------------------------------------------
hadamard                      8          8          8          8
ry_encoding                   8          8          8          8
rz_equivariant                8          8          8          8
rz_entanglement               8          6         12          0
ry_entanglement               0          0          0         12
cnot                          0         12          0         18
crz                           4          0          0          0
cz                            0          0          6          0
total_single_qubit           32         30         36         36
total_two_qubit               8         12          6         18
total                        40         42         42         54


---
## 16. Resource Summary

`resource_summary()` combines gate counts, properties, symmetry info, and hardware requirements into a comprehensive dictionary.

In [59]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
summary = enc.resource_summary()

print("=== Top-level keys ===")
for key in summary:
    val = summary[key]
    if isinstance(val, dict):
        print(f"  {key}: <dict with {len(val)} entries>")
    else:
        print(f"  {key}: {val}")

=== Top-level keys ===
  n_qubits: 4
  n_features: 4
  depth: 8
  reps: 2
  symmetry: rotation
  entanglement: linear
  feature_map: angle
  entanglement_pairs: [(0, 1), (1, 2), (2, 3)]
  n_entanglement_pairs: 3
  symmetry_info: <dict with 6 entries>
  gate_counts: <dict with 11 entries>
  is_entangling: True
  simulability: not_simulable
  trainability_estimate: 0.55
  hardware_requirements: <dict with 3 entries>
  memory_estimate: <dict with 8 entries>


In [60]:
# Hardware requirements
hw = summary['hardware_requirements']
print("=== Hardware Requirements ===")
for key, value in hw.items():
    print(f"  {key}: {value}")

=== Hardware Requirements ===
  connectivity: linear
  native_gates: ['H', 'RY', 'RZ', 'CRZ']
  min_two_qubit_gate_fidelity: 0.99


---
## 17. Module-Level Utility Functions

The module provides several utility functions that work without instantiating an encoding.

In [61]:
from encoding_atlas.encodings.symmetry_inspired_feature_map import (
    get_supported_symmetries,
    estimate_circuit_resources,
    estimate_memory_usage,
)

# List all supported symmetry types
symmetries = get_supported_symmetries()
print(f"Supported symmetries: {symmetries}")
assert symmetries == ['cyclic', 'full', 'reflection', 'rotation']  # sorted

Supported symmetries: ['cyclic', 'full', 'reflection', 'rotation']


In [62]:
# Estimate resources without creating an encoding
resources = estimate_circuit_resources(n_features=8, symmetry='rotation', reps=3, entanglement='linear')
print("=== estimate_circuit_resources(n=8, rotation, reps=3, linear) ===")
for key, value in resources.items():
    print(f"  {key}: {value}")

=== estimate_circuit_resources(n=8, rotation, reps=3, linear) ===
  n_qubits: 8
  estimated_depth: 12
  estimated_two_qubit_gates: 24
  entanglement_depth_per_rep: 1
  n_entanglement_pairs: 7


In [63]:
# Estimate memory usage
mem = estimate_memory_usage(n_features=8, symmetry='rotation', reps=2, include_state_vector=True)
print("=== estimate_memory_usage(n=8, rotation, reps=2) ===")
for key, value in mem.items():
    if 'bytes' in key or 'mb' in key:
        print(f"  {key}: {value:,.0f}" if isinstance(value, (int, float)) else f"  {key}: {value}")
    else:
        print(f"  {key}: {value}")

=== estimate_memory_usage(n=8, rotation, reps=2) ===
  instance_bytes: 410
  entanglement_pairs_bytes: 560
  symmetry_params_bytes: 744
  total_instance_bytes: 1,714
  circuit_angles_bytes: 520
  total_per_circuit_bytes: 4,616
  n_entanglement_pairs: 7
  recommended_max_features: 500
  state_vector_bytes: 4,096
  state_vector_mb: 0


In [64]:
# Memory estimation with different entanglement patterns
print("=== Memory scaling with entanglement ===")
for ent in ['linear', 'circular', 'full']:
    mem = estimate_memory_usage(n_features=10, symmetry='cyclic', entanglement=ent)
    print(f"  {ent:10s}: instance={mem['total_instance_bytes']:,} bytes, "
          f"recommended_max_features={mem['recommended_max_features']}")

=== Memory scaling with entanglement ===
  linear    : instance=1,458 bytes, recommended_max_features=500
  circular  : instance=1,530 bytes, recommended_max_features=500
  full      : instance=4,050 bytes, recommended_max_features=50


---
## 18. Analysis Tools — Simulability

Check whether an encoding can be efficiently simulated classically.

In [65]:
from encoding_atlas.analysis import (
    check_simulability,
    get_simulability_reason,
    is_clifford_circuit,
    is_matchgate_circuit,
)

# Non-entangling → simulable (product states)
enc_product = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', entanglement='none')
result = check_simulability(enc_product, detailed=True)
print(f"=== Product state (entanglement='none') ===")
print(f"  simulability_class: {result['simulability_class']}")
print(f"  reason: {result['reason']}")
print(f"  is_simulable: {result['is_simulable']}")

print(f"\n  Quick reason: {get_simulability_reason(enc_product)}")

=== Product state (entanglement='none') ===
  simulability_class: simulable
  reason: Encoding produces only product states (no entanglement)
  is_simulable: True

  Quick reason: Simulable: Encoding produces only product states (no entanglement)


C:\Users\ashut\AppData\Local\Temp\ipykernel_63124\1747577785.py:9: UserWarning: When symmetry='rotation', the entanglement parameter ('none') is overridden. Rotation symmetry always uses coordinate pairs [(0,1), (2,3), ...] for entanglement to maintain SO(2) equivariance. The 'none' entanglement pattern will be ignored. To suppress this warning, use entanglement='linear' or choose a different symmetry type.
  enc_product = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', entanglement='none')


In [66]:
# Entangling → not simulable
enc_entangling = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', entanglement='linear')
result = check_simulability(enc_entangling, detailed=True)
print(f"=== Entangling (entanglement='linear') ===")
print(f"  simulability_class: {result['simulability_class']}")
print(f"  reason: {result['reason']}")
print(f"  is_simulable: {result['is_simulable']}")
print(f"\n  Details:")
for key, value in result['details'].items():
    print(f"    {key}: {value}")
print(f"\n  Recommendations:")
for rec in result['recommendations']:
    print(f"    - {rec}")

=== Entangling (entanglement='linear') ===
  simulability_class: conditionally_simulable
  reason: Linear entanglement structure may allow tensor network simulation if entanglement entropy is bounded
  is_simulable: False

  Details:
    is_entangling: True
    is_clifford: False
    is_matchgate: False
    entanglement_pattern: linear
    two_qubit_gate_count: 8
    n_qubits: 4
    n_features: 4
    declared_simulability: not_simulable
    encoding_name: SymmetryInspiredFeatureMap
    has_non_clifford_gates: False
    has_t_gates: False
    has_parameterized_rotations: False

  Recommendations:
    - Statevector simulation feasible (4 qubits, ~256 bytes memory)
    - Consider MPS (Matrix Product State) simulation
    - May be efficient if entanglement entropy is bounded
    - Tensor network methods scale with bond dimension


In [67]:
# Clifford and matchgate checks
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation')
print(f"is_clifford_circuit:  {is_clifford_circuit(enc)}")
print(f"is_matchgate_circuit: {is_matchgate_circuit(enc)}")

is_clifford_circuit:  False
is_matchgate_circuit: False


---
## 19. Analysis Tools — Expressibility

Measures how well the encoding explores the Hilbert space compared to Haar-random states. Score in [0, 1]; higher = more expressive.

In [68]:
from encoding_atlas.analysis import compute_expressibility

# Small encoding for fast demo
enc = SymmetryInspiredFeatureMap(n_features=2, symmetry='reflection', reps=2)

expr = compute_expressibility(enc, n_samples=1000, seed=42)
print(f"Expressibility score: {expr:.4f}")

c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=2.758, max=4.863). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 2 values outside range (min=4.382, max=5.395). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=0.5917, max=6.13). This may produce unexpected rotation angl

c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=2.96, max=4.2). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 2 values outside range (min=3.551, max=4.807). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 2 values outside range (min=3.478, max=3.988). This may produce unexpected rotation angles.

Expressibility score: 0.9618


c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=0.4332, max=3.728). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 2 values outside range (min=5.95, max=6.215). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=1.721, max=4.366). This may produce unexpected rotation angl

In [69]:
# Detailed result with distributions
expr_detailed = compute_expressibility(
    enc, n_samples=1000, seed=42, return_distributions=True
)

print("=== Detailed Expressibility Result ===")
print(f"  expressibility:       {expr_detailed['expressibility']:.4f}")
print(f"  kl_divergence:        {expr_detailed['kl_divergence']:.4f}")
print(f"  n_samples:            {expr_detailed['n_samples']}")
print(f"  n_bins:               {expr_detailed['n_bins']}")
print(f"  convergence_estimate: {expr_detailed['convergence_estimate']:.4f}")
print(f"  mean_fidelity:        {expr_detailed['mean_fidelity']:.4f}")
print(f"  std_fidelity:         {expr_detailed['std_fidelity']:.4f}")

=== Detailed Expressibility Result ===
  expressibility:       0.9618
  kl_divergence:        0.3819
  n_samples:            1000
  n_bins:               75
  convergence_estimate: 0.0370
  mean_fidelity:        0.4030
  std_fidelity:         0.2589


In [70]:
# Compare expressibility across symmetry types
print("=== Expressibility by symmetry (n=4, reps=2) ===")
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym, reps=2)
    expr = compute_expressibility(enc, n_samples=500, seed=42)
    print(f"  {sym:12s}: {expr:.4f}")

=== Expressibility by symmetry (n=4, reps=2) ===


c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=2.758, max=5.395). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=0.5917, max=6.13). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=0.805, max=5.823). This may produce unexpected rotation angl

  rotation    : 0.9214
  cyclic      : 0.9372
  reflection  : 0.9052
  full        : 0.9978


In [71]:
# Expressibility increases with reps
print("=== Expressibility vs reps (rotation, n=4) ===")
for reps in [1, 2, 3, 5]:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=reps)
    expr = compute_expressibility(enc, n_samples=500, seed=42)
    print(f"  reps={reps}: {expr:.4f}")

=== Expressibility vs reps (rotation, n=4) ===
  reps=1: 0.9625
  reps=2: 0.9214
  reps=3: 0.9708
  reps=5: 0.9669


---
## 20. Analysis Tools — Entanglement Capability

Measures the average entanglement generated by the encoding across random inputs. Uses Meyer-Wallach or Scott entanglement measures.

In [72]:
from encoding_atlas.analysis import (
    compute_entanglement_capability,
    compute_meyer_wallach,
)

# Non-entangling encoding → zero entanglement (use cyclic to avoid rotation warning)
enc_none = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', entanglement='none')
ent_none = compute_entanglement_capability(enc_none, n_samples=100, seed=42)
print(f"entanglement='none': entanglement_capability = {ent_none:.2e}")
assert ent_none < 1e-10, "Product states have (near-)zero entanglement"
print("Product state: effectively zero ✓")

entanglement='none': entanglement_capability = 6.15e-15
Product state: effectively zero ✓


In [73]:
# Entangling encoding → non-zero entanglement
enc_ent = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', entanglement='linear')
ent_result = compute_entanglement_capability(
    enc_ent, n_samples=200, seed=42, return_details=True
)

print("=== Entanglement Capability (rotation, linear) ===")
print(f"  capability:  {ent_result['entanglement_capability']:.4f}")
print(f"  std_error:   {ent_result['std_error']:.4f}")
print(f"  n_samples:   {ent_result['n_samples']}")
print(f"  measure:     {ent_result['measure']}")
print(f"  per_qubit:   {ent_result['per_qubit_entanglement']}")

=== Entanglement Capability (rotation, linear) ===
  capability:  0.1307
  std_error:   0.0099
  n_samples:   200
  measure:     meyer_wallach
  per_qubit:   [0.06325568 0.06325568 0.06748921 0.06748921]


In [74]:
# Compare entanglement across symmetry types
print("=== Entanglement by symmetry (n=4, reps=2, linear) ===")
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym, reps=2)
    ent = compute_entanglement_capability(enc, n_samples=200, seed=42)
    print(f"  {sym:12s}: {ent:.4f}")

=== Entanglement by symmetry (n=4, reps=2, linear) ===
  rotation    : 0.1307
  cyclic      : 0.1480
  reflection  : 0.3000
  full        : 0.6853


In [75]:
# Meyer-Wallach on a specific state
state = simulate_encoding_statevector(enc_ent, np.array([0.5, 1.0, 1.5, 2.0]))
mw = compute_meyer_wallach(state, n_qubits=4)
print(f"Meyer-Wallach for specific input: {mw:.4f}")

Meyer-Wallach for specific input: 0.0599


---
## 21. Analysis Tools — Trainability

Estimates trainability by analyzing gradient variance. Low variance indicates barren plateaus (vanishing gradients).

In [76]:
from encoding_atlas.analysis import estimate_trainability, detect_barren_plateau

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)

# Quick trainability score
train_score = estimate_trainability(enc, n_samples=100, seed=42)
print(f"Trainability score: {train_score:.4f}")

c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=2.758, max=6.434). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 4 values outside range (min=4.328, max=5.395). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=1.187, max=5.395). This may produce unexpected rotation angl

Trainability score: 0.0924


c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=0.0773, max=4.425). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=1.856, max=6.757). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=1.856, max=3.615). This may produce unexpected rotation ang

In [77]:
# Detailed result
train_detailed = estimate_trainability(enc, n_samples=100, seed=42, return_details=True)

print("=== Detailed Trainability ===")
print(f"  trainability_estimate:   {train_detailed['trainability_estimate']:.4f}")
print(f"  gradient_variance:       {train_detailed['gradient_variance']:.6f}")
print(f"  barren_plateau_risk:     {train_detailed['barren_plateau_risk']}")
print(f"  effective_dimension:     {train_detailed['effective_dimension']}")
print(f"  n_samples:               {train_detailed['n_samples']}")
print(f"  n_successful_samples:    {train_detailed['n_successful_samples']}")
print(f"  n_failed_samples:        {train_detailed['n_failed_samples']}")
print(f"  per_parameter_variance:  {train_detailed['per_parameter_variance']}")

=== Detailed Trainability ===
  trainability_estimate:   0.0924
  gradient_variance:       0.007206
  barren_plateau_risk:     low
  effective_dimension:     4.0
  n_samples:               100
  n_successful_samples:    100
  n_failed_samples:        0
  per_parameter_variance:  [0.00652896 0.00834846 0.00600487 0.00794042]


In [78]:
# Barren plateau detection
bp_risk = detect_barren_plateau(
    train_detailed['gradient_variance'],
    n_qubits=enc.n_qubits,
    n_params=enc.n_features,
)
print(f"Barren plateau risk: {bp_risk}")

Barren plateau risk: low


---
## 22. Built-in `measure_trainability()` Method

SymmetryInspiredFeatureMap provides its own `measure_trainability()` method that appends a trainable RY layer after the encoding circuit and empirically measures gradient variance. This is separate from the analysis module's `estimate_trainability()` and provides a more encoding-specific measurement.

In [79]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
x = np.array([0.5, 1.0, 1.5, 2.0])

result = enc.measure_trainability(x, n_samples=50, seed=42)
print("=== measure_trainability() result ===")
print(f"  gradient_variance:      {result['gradient_variance']:.6f}")
print(f"  gradient_mean:          {result['gradient_mean']:.6f}")
print(f"  gradient_std:           {result['gradient_std']:.6f}")
print(f"  empirical_trainability: {result['empirical_trainability']:.4f}")
print(f"  heuristic_trainability: {result['heuristic_trainability']:.4f}")
print(f"  n_samples:              {result['n_samples']}")
print(f"  cost_observable:        {result['cost_observable']}")
print(f"  gradients shape:        {result['gradients'].shape}")

=== measure_trainability() result ===
  gradient_variance:      0.042968
  gradient_mean:          0.184851
  gradient_std:           0.207287
  empirical_trainability: 0.4297
  heuristic_trainability: 0.5500
  n_samples:              50
  cost_observable:        local
  gradients shape:        (50,)


In [80]:
# Local vs global cost observable
local = enc.measure_trainability(x, n_samples=50, cost_observable='local', seed=42)
global_ = enc.measure_trainability(x, n_samples=50, cost_observable='global', seed=42)

print("=== Local vs Global Cost Observable ===")
print(f"  Local  gradient_variance: {local['gradient_variance']:.6f}")
print(f"  Global gradient_variance: {global_['gradient_variance']:.6f}")
print("  (Local cost functions typically have better trainability)")

=== Local vs Global Cost Observable ===
  Local  gradient_variance: 0.042968
  Global gradient_variance: 0.018185
  (Local cost functions typically have better trainability)


In [81]:
# Compare trainability across symmetry types
print("=== measure_trainability by symmetry ===")
x = np.array([0.5, 1.0, 1.5, 2.0])
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym, reps=2)
    result = enc.measure_trainability(x, n_samples=50, seed=42)
    print(f"  {sym:12s}: variance={result['gradient_variance']:.6f}, "
          f"empirical={result['empirical_trainability']:.4f}")

=== measure_trainability by symmetry ===
  rotation    : variance=0.042968, empirical=0.4297
  cyclic      : variance=0.234445, empirical=1.0000
  reflection  : variance=0.310451, empirical=1.0000
  full        : variance=0.033052, empirical=0.3305


In [82]:
# Seed reproducibility
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
r1 = enc.measure_trainability(x, n_samples=30, seed=42)
r2 = enc.measure_trainability(x, n_samples=30, seed=42)
assert np.allclose(r1['gradients'], r2['gradients'])
print("Same seed → identical gradients ✓")

r3 = enc.measure_trainability(x, n_samples=30, seed=99)
assert not np.allclose(r1['gradients'], r3['gradients'])
print("Different seed → different gradients ✓")

Same seed → identical gradients ✓
Different seed → different gradients ✓


In [83]:
# Validation of measure_trainability parameters
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
x = np.array([0.5, 1.0, 1.5, 2.0])

# n_samples must be >= 10
try:
    enc.measure_trainability(x, n_samples=5)
except ValueError as e:
    print(f"  n_samples=5: ValueError ✓")

# n_samples must be integer
try:
    enc.measure_trainability(x, n_samples=50.0)
except TypeError as e:
    print(f"  n_samples=50.0: TypeError ✓")

# cost_observable must be 'local' or 'global'
try:
    enc.measure_trainability(x, cost_observable='invalid')
except ValueError as e:
    print(f"  cost_observable='invalid': ValueError ✓")

# seed must be int or None
try:
    enc.measure_trainability(x, seed=3.14)
except TypeError as e:
    print(f"  seed=3.14: TypeError ✓")

# numpy integer seed is accepted
result = enc.measure_trainability(x, n_samples=10, seed=np.int64(42))
print(f"  seed=np.int64(42): accepted ✓")

  n_samples=5: ValueError ✓
  n_samples=50.0: TypeError ✓
  cost_observable='invalid': ValueError ✓
  seed=3.14: TypeError ✓
  seed=np.int64(42): accepted ✓


---
## 23. Analysis Tools — Resource Counting

The analysis module provides additional resource analysis beyond the built-in methods.

In [84]:
from encoding_atlas.analysis import (
    count_resources,
    get_resource_summary,
    get_gate_breakdown,
    compare_resources,
    estimate_execution_time,
)

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)

# Count resources
resources = count_resources(enc)
print("=== count_resources() ===")
for key, value in resources.items():
    print(f"  {key}: {value}")

=== count_resources() ===
  n_qubits: 4
  depth: 8
  gate_count: 40
  single_qubit_gates: 32
  two_qubit_gates: 8
  parameter_count: 0
  cnot_count: 0
  cz_count: 0
  t_gate_count: 0
  hadamard_count: 8
  rotation_gates: 0
  two_qubit_ratio: 0.2
  gates_per_qubit: 10.0
  encoding_name: SymmetryInspiredFeatureMap
  is_data_dependent: False


In [85]:
# Detailed gate breakdown
detailed = count_resources(enc, detailed=True)
print("=== count_resources(detailed=True) ===")
for key, value in detailed.items():
    print(f"  {key}: {value}")

=== count_resources(detailed=True) ===
  rx: 0
  ry: 0
  rz: 0
  h: 8
  x: 0
  y: 0
  z: 0
  s: 0
  t: 0
  cnot: 0
  cx: 0
  cz: 0
  swap: 0
  total_single_qubit: 32
  total_two_qubit: 8
  total: 40
  encoding_name: SymmetryInspiredFeatureMap


In [86]:
# Compare resources across different configurations
encs = [
    SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2),
    SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', reps=2),
    SymmetryInspiredFeatureMap(n_features=4, symmetry='reflection', reps=2),
    SymmetryInspiredFeatureMap(n_features=4, symmetry='full', reps=2),
]
comparison = compare_resources(encs)
print("=== compare_resources() ===")
for metric, values in comparison.items():
    print(f"  {metric:25s}: {values}")

=== compare_resources() ===
  n_qubits                 : [4, 4, 4, 4]
  depth                    : [8, 10, 10, 10]
  gate_count               : [40, 42, 42, 54]
  single_qubit_gates       : [32, 30, 36, 36]
  two_qubit_gates          : [8, 12, 6, 18]
  parameter_count          : [0, 0, 0, 0]
  two_qubit_ratio          : [0.2, 0.2857142857142857, 0.14285714285714285, 0.3333333333333333]
  gates_per_qubit          : [10.0, 10.5, 10.5, 13.5]
  encoding_name            : ['SymmetryInspiredFeatureMap', 'SymmetryInspiredFeatureMap', 'SymmetryInspiredFeatureMap', 'SymmetryInspiredFeatureMap']


In [87]:
# Estimate execution time on quantum hardware
exec_time = estimate_execution_time(
    enc,
    single_qubit_gate_time_us=0.02,   # 20 ns
    two_qubit_gate_time_us=0.2,        # 200 ns
    measurement_time_us=1.0,           # 1 μs
)
print("=== estimate_execution_time() ===")
for key, value in exec_time.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f} μs")
    else:
        print(f"  {key}: {value}")

=== estimate_execution_time() ===
  serial_time_us: 3.2400 μs
  estimated_time_us: 2.6000 μs
  single_qubit_time_us: 0.6400 μs
  two_qubit_time_us: 1.6000 μs
  measurement_time_us: 1.0000 μs
  parallelization_factor: 0.5000 μs


---
## 24. Low-Level Analysis Utilities

The `encoding_atlas.analysis` module provides building-block functions for custom analysis.

In [88]:
from encoding_atlas.analysis import (
    simulate_encoding_statevector,
    simulate_encoding_statevectors_batch,
    compute_fidelity,
    compute_purity,
    compute_linear_entropy,
    compute_von_neumann_entropy,
    partial_trace_single_qubit,
    partial_trace_subsystem,
    validate_encoding_for_analysis,
    generate_random_parameters,
    create_rng,
)

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
x = np.array([0.5, 1.0, 1.5, 2.0])

# Statevector simulation
state = simulate_encoding_statevector(enc, x)
print(f"State shape: {state.shape}, norm: {np.linalg.norm(state):.10f}")

State shape: (16,), norm: 1.0000000000


In [89]:
# Batch statevector simulation
X_batch = np.array([
    [0.0, 0.0, 0.0, 0.0],
    [0.5, 1.0, 1.5, 2.0],
    [np.pi, np.pi, np.pi, np.pi],
])
states = simulate_encoding_statevectors_batch(enc, X_batch)
print(f"Batch: {len(states)} states, each shape {states[0].shape}")
for i, s in enumerate(states):
    print(f"  State {i} norm: {np.linalg.norm(s):.10f}")

Batch: 3 states, each shape (16,)
  State 0 norm: 1.0000000000
  State 1 norm: 1.0000000000
  State 2 norm: 1.0000000000


In [90]:
# Fidelity between states
state1 = simulate_encoding_statevector(enc, np.array([0.5, 1.0, 1.5, 2.0]))
state2 = simulate_encoding_statevector(enc, np.array([0.5, 1.0, 1.5, 2.0]))  # same input
state3 = simulate_encoding_statevector(enc, np.array([2.0, 0.5, 0.1, 3.0]))  # different

fid_same = compute_fidelity(state1, state2)
fid_diff = compute_fidelity(state1, state3)
print(f"Fidelity (same input):      {fid_same:.6f}")
print(f"Fidelity (different input): {fid_diff:.6f}")
assert np.isclose(fid_same, 1.0)
print("Same input → fidelity = 1.0 ✓")

Fidelity (same input):      1.000000
Fidelity (different input): 0.059919
Same input → fidelity = 1.0 ✓


In [91]:
# Partial trace — reduced density matrix
rho_0 = partial_trace_single_qubit(state1, n_qubits=4, keep_qubit=0)
print(f"Single-qubit density matrix shape: {rho_0.shape}")
print(f"Trace: {np.trace(rho_0).real:.6f}")  # Should be 1
print(f"Purity: {compute_purity(rho_0):.6f}")
print(f"Linear entropy: {compute_linear_entropy(rho_0):.6f}")
print(f"Von Neumann entropy: {compute_von_neumann_entropy(rho_0):.6f} bits")

Single-qubit density matrix shape: (2, 2)
Trace: 1.000000
Purity: 0.942778
Linear entropy: 0.057222
Von Neumann entropy: 0.191777 bits


In [92]:
# Subsystem partial trace
rho_01 = partial_trace_subsystem(state1, n_qubits=4, keep_qubits=[0, 1])
print(f"2-qubit subsystem density matrix shape: {rho_01.shape}")
print(f"Trace: {np.trace(rho_01).real:.6f}")
print(f"Purity: {compute_purity(rho_01):.6f}")

2-qubit subsystem density matrix shape: (4, 4)
Trace: 1.000000
Purity: 1.000000


In [93]:
# Validation utility
validate_encoding_for_analysis(enc)
print("validate_encoding_for_analysis: passed ✓")

validate_encoding_for_analysis: passed ✓


In [94]:
# Random parameter generation
params = generate_random_parameters(enc, n_samples=5, seed=42)
print(f"Random params shape: {params.shape}")
print(params)

# From integer (no encoding needed)
params_int = generate_random_parameters(4, n_samples=3, param_min=0, param_max=1, seed=42)
print(f"\nFrom integer: shape {params_int.shape}")

Random params shape: (5, 4)
[[4.86290927 2.75755456 5.39472984 4.38169255]
 [0.59173373 6.13001603 4.78238179 4.93898769]
 [0.80496169 2.82985831 2.3297927  5.82303616]
 [4.04552386 5.16956368 2.78605358 1.427783  ]
 [3.48455899 0.40097565 5.20016002 3.96886447]]

From integer: shape (3, 4)


In [95]:
# Reproducible RNG
rng1 = create_rng(seed=42)
rng2 = create_rng(seed=42)
assert np.array_equal(rng1.random(5), rng2.random(5))
print("create_rng(seed=42) is reproducible ✓")

create_rng(seed=42) is reproducible ✓


---
## 25. Capability Protocols

The library uses Python's structural subtyping (PEP 544) to define optional capabilities that encodings may implement.

In [96]:
from encoding_atlas.core.protocols import (
    ResourceAnalyzable,
    EntanglementQueryable,
    is_resource_analyzable,
    is_entanglement_queryable,
)

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)

print(f"is_resource_analyzable:     {is_resource_analyzable(enc)}")
print(f"is_entanglement_queryable:  {is_entanglement_queryable(enc)}")

# SymmetryInspiredFeatureMap implements both protocols
if is_resource_analyzable(enc):
    summary = enc.resource_summary()
    breakdown = enc.gate_count_breakdown()
    print(f"\nResourceAnalyzable: resource_summary() has {len(summary)} entries ✓")
    print(f"ResourceAnalyzable: gate_count_breakdown() has {len(breakdown)} entries ✓")

if is_entanglement_queryable(enc):
    pairs = enc.get_entanglement_pairs()
    print(f"EntanglementQueryable: {len(pairs)} entanglement pairs ✓")

is_resource_analyzable:     True
is_entanglement_queryable:  True

ResourceAnalyzable: resource_summary() has 16 entries ✓
ResourceAnalyzable: gate_count_breakdown() has 11 entries ✓
EntanglementQueryable: 3 entanglement pairs ✓


In [97]:
# Generic analysis code that works with any encoding via protocols
def analyze_any_encoding(enc):
    """Works with any encoding that implements the right protocols."""
    info = {"name": type(enc).__name__, "n_features": enc.n_features}

    if is_resource_analyzable(enc):
        info["gate_count"] = enc.resource_summary().get("gate_count_breakdown", {})

    if is_entanglement_queryable(enc):
        info["n_pairs"] = len(enc.get_entanglement_pairs())
    else:
        info["n_pairs"] = "N/A (not queryable)"

    return info

result = analyze_any_encoding(enc)
print(f"Generic analysis: {result}")

Generic analysis: {'name': 'SymmetryInspiredFeatureMap', 'n_features': 4, 'gate_count': {}, 'n_pairs': 3}


---
## 26. Registry System

SymmetryInspiredFeatureMap is registered under multiple names in the global encoding registry.

In [98]:
from encoding_atlas import get_encoding, list_encodings

# List all registered encodings
all_encodings = list_encodings()
print(f"Total registered encodings: {len(all_encodings)}")
print(f"\nAll encodings: {all_encodings}")

Total registered encodings: 26

All encodings: ['amplitude', 'angle', 'angle_ry', 'basis', 'covariant', 'covariant_feature_map', 'cyclic_equivariant', 'cyclic_equivariant_feature_map', 'data_reuploading', 'hamiltonian', 'hamiltonian_encoding', 'hardware_efficient', 'higher_order_angle', 'iqp', 'pauli_feature_map', 'qaoa', 'qaoa_encoding', 'so2_equivariant', 'so2_equivariant_feature_map', 'swap_equivariant', 'swap_equivariant_feature_map', 'symmetry_inspired', 'symmetry_inspired_feature_map', 'trainable', 'trainable_encoding', 'zz_feature_map']


In [99]:
# Create via registry using any registered name
enc1 = get_encoding("symmetry_inspired_feature_map", n_features=4, symmetry='rotation')
enc2 = get_encoding("symmetry_inspired", n_features=4, symmetry='rotation')
enc3 = get_encoding("covariant_feature_map", n_features=4, symmetry='rotation')  # backwards compat
enc4 = get_encoding("covariant", n_features=4, symmetry='rotation')               # backwards compat

assert enc1 == enc2 == enc3 == enc4
print("All registry aliases produce equal encodings ✓")
print(f"  'symmetry_inspired_feature_map' ✓")
print(f"  'symmetry_inspired' ✓")
print(f"  'covariant_feature_map' (backward compat) ✓")
print(f"  'covariant' (backward compat) ✓")

All registry aliases produce equal encodings ✓
  'symmetry_inspired_feature_map' ✓
  'symmetry_inspired' ✓
  'covariant_feature_map' (backward compat) ✓
  'covariant' (backward compat) ✓


In [100]:
# RegistryError for unknown names
from encoding_atlas.core.exceptions import RegistryError

try:
    get_encoding("nonexistent_encoding", n_features=4)
except RegistryError as e:
    print(f"RegistryError: {e}")

RegistryError: Unknown encoding 'nonexistent_encoding'. Available encodings: amplitude, angle, angle_ry, basis, covariant, covariant_feature_map, cyclic_equivariant, cyclic_equivariant_feature_map, data_reuploading, hamiltonian, hamiltonian_encoding, hardware_efficient, higher_order_angle, iqp, pauli_feature_map, qaoa, qaoa_encoding, so2_equivariant, so2_equivariant_feature_map, swap_equivariant, swap_equivariant_feature_map, symmetry_inspired, symmetry_inspired_feature_map, trainable, trainable_encoding, zz_feature_map


---
## 27. Deprecated Alias: CovariantFeatureMap

`CovariantFeatureMap` is a deprecated alias for `SymmetryInspiredFeatureMap`. It emits a `DeprecationWarning` and will be removed in v1.0.0.

In [101]:
from encoding_atlas import CovariantFeatureMap
import warnings

with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    enc_deprecated = CovariantFeatureMap(n_features=4, symmetry='rotation')
    if w:
        print(f"DeprecationWarning: {w[0].message}")

# It's a subclass of SymmetryInspiredFeatureMap
print(f"\ntype: {type(enc_deprecated).__name__}")
print(f"isinstance(SymmetryInspiredFeatureMap): "
      f"{isinstance(enc_deprecated, SymmetryInspiredFeatureMap)}")

# Works identically
x = np.array([0.5, 1.0, 1.5, 2.0])
circuit = enc_deprecated.get_circuit(x, backend='pennylane')
print(f"Circuit generated: {callable(circuit)} ✓")


type: CovariantFeatureMap
isinstance(SymmetryInspiredFeatureMap): True
Circuit generated: True ✓


---
## 28. Equality, Hashing & Serialization

SymmetryInspiredFeatureMap supports equality comparison, hashing (usable in sets/dicts), and pickle serialization.

In [102]:
# --- Equality ---
enc_a = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
enc_b = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
enc_c = SymmetryInspiredFeatureMap(n_features=4, symmetry='cyclic', reps=2)

print("=== Equality ===")
print(f"  Same params (a == b): {enc_a == enc_b}")
print(f"  Diff symmetry (a == c): {enc_a == enc_c}")
assert enc_a == enc_b
assert enc_a != enc_c

# Different reps
enc_d = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=3)
assert enc_a != enc_d
print(f"  Diff reps (a == d): {enc_a == enc_d}")

# Comparison with non-SymmetryInspiredFeatureMap returns NotImplemented
from encoding_atlas import AngleEncoding
enc_angle = AngleEncoding(n_features=4)
print(f"  Different type (a == AngleEncoding): {enc_a == enc_angle}")

=== Equality ===
  Same params (a == b): True
  Diff symmetry (a == c): False
  Diff reps (a == d): False
  Different type (a == AngleEncoding): False


In [103]:
# --- Hashing ---
print("=== Hashing ===")
print(f"  hash(a) == hash(b): {hash(enc_a) == hash(enc_b)}")
assert hash(enc_a) == hash(enc_b)

# Usable as dict keys and in sets
encoding_set = {enc_a, enc_b, enc_c}
print(f"  Set {{a, b, c}} has {len(encoding_set)} elements (a == b, so 2)")
assert len(encoding_set) == 2

encoding_dict = {enc_a: "rotation", enc_c: "cyclic"}
print(f"  Dict lookup enc_b → {encoding_dict[enc_b]!r} (same hash as a)")

=== Hashing ===
  hash(a) == hash(b): True
  Set {a, b, c} has 2 elements (a == b, so 2)
  Dict lookup enc_b → 'rotation' (same hash as a)


In [104]:
# --- Pickle Serialization ---
import pickle

enc_original = SymmetryInspiredFeatureMap(
    n_features=6, symmetry='reflection', reps=3,
    entanglement='circular', feature_map='polynomial'
)
# Force properties computation before pickling
_ = enc_original.properties

# Serialize and deserialize
data = pickle.dumps(enc_original)
enc_restored = pickle.loads(data)

print("=== Pickle Serialization ===")
print(f"  Equal after pickle: {enc_original == enc_restored}")
print(f"  symmetry matches:   {enc_original.symmetry == enc_restored.symmetry}")
print(f"  reps matches:       {enc_original.reps == enc_restored.reps}")
print(f"  entanglement:       {enc_original.entanglement == enc_restored.entanglement}")
print(f"  feature_map:        {enc_original.feature_map == enc_restored.feature_map}")

assert enc_original == enc_restored

# Properties are recomputed after unpickling (not carried over)
props = enc_restored.properties
print(f"  Properties recomputed: n_qubits={props.n_qubits}, depth={props.depth} ✓")

# Circuit still works after unpickling
x_test = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
circuit = enc_restored.get_circuit(x_test, backend='pennylane')
print(f"  Circuit works after unpickle: {callable(circuit)} ✓")

=== Pickle Serialization ===
  Equal after pickle: True
  symmetry matches:   True
  reps matches:       True
  entanglement:       True
  feature_map:        True
  Properties recomputed: n_qubits=6, depth=15 ✓
  Circuit works after unpickle: True ✓


---
## 29. String Representation

The `__repr__` method shows all constructor parameters for easy reproduction.

In [105]:
enc = SymmetryInspiredFeatureMap(
    n_features=6, symmetry='reflection', reps=3,
    entanglement='circular', feature_map='polynomial', include_barriers=False
)
print(repr(enc))

# repr contains all parameter names and values
r = repr(enc)
assert 'SymmetryInspiredFeatureMap' in r
assert 'n_features=6' in r
assert "symmetry='reflection'" in r
assert 'reps=3' in r
assert "entanglement='circular'" in r
assert "feature_map='polynomial'" in r
assert 'include_barriers=False' in r
print("\nAll parameters present in repr ✓")

SymmetryInspiredFeatureMap(n_features=6, symmetry='reflection', reps=3, entanglement='circular', feature_map='polynomial', include_barriers=False)

All parameters present in repr ✓


---
## 30. Thread Safety

SymmetryInspiredFeatureMap is designed for safe concurrent use. Circuit generation is stateless, and properties use double-checked locking.

In [106]:
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)

# Concurrent circuit generation
def generate_circuit(idx):
    x = np.random.RandomState(idx).uniform(0, np.pi, 4)
    circuit = enc.get_circuit(x, backend='pennylane')
    return idx, callable(circuit)

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(generate_circuit, i) for i in range(20)]
    results = [f.result() for f in as_completed(futures)]

all_ok = all(ok for _, ok in results)
print(f"20 concurrent circuit generations: all callable = {all_ok} ✓")

20 concurrent circuit generations: all callable = True ✓


In [107]:
# Concurrent property access (tests double-checked locking)
enc_fresh = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)
properties_list = []

def access_properties():
    props = enc_fresh.properties
    properties_list.append(id(props))

threads = [threading.Thread(target=access_properties) for _ in range(30)]
for t in threads:
    t.start()
for t in threads:
    t.join()

# All threads should get the same cached object
assert len(set(properties_list)) == 1
print(f"30 threads accessed properties: all same object (id={properties_list[0]}) ✓")

30 threads accessed properties: all same object (id=2595894987216) ✓


---
## 31. Logging & Debugging

SymmetryInspiredFeatureMap uses Python's standard logging module. Enable it to see internal operations.

In [108]:
import logging

# Enable debug logging for the symmetry module
logger = logging.getLogger('encoding_atlas.encodings.symmetry_inspired_feature_map')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

print("Creating encoding with debug logging...")
enc_debug = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=1)

print("\nGenerating circuit with debug logging...")
x = np.array([0.5, 1.0, 1.5, 2.0])
_ = enc_debug.get_circuit(x, backend='pennylane')

# Clean up
logger.removeHandler(handler)
logger.setLevel(logging.WARNING)

encoding_atlas.encodings.symmetry_inspired_feature_map - DEBUG - SymmetryInspiredFeatureMap initialized: n_features=4, symmetry='rotation', reps=1, entanglement='linear', feature_map='angle', n_pairs=3
encoding_atlas.encodings.symmetry_inspired_feature_map - DEBUG - Generating circuit: backend='pennylane', input_shape=(4,)
encoding_atlas.encodings.symmetry_inspired_feature_map - DEBUG - Circuit generated successfully for backend='pennylane'


Creating encoding with debug logging...

Generating circuit with debug logging...


---
## 32. Numerical Stability & Edge Cases

The encoding handles extreme input values gracefully using `np.hypot` for overflow-safe radius calculations.

In [109]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=1)

# Very small values (near machine epsilon)
x_tiny = np.array([1e-15, 1e-18, 1e-15, 1e-18])
circuit = enc.get_circuit(x_tiny, backend='pennylane')
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def run_tiny():
    circuit()
    return qml.state()

state = run_tiny()
assert np.isclose(np.linalg.norm(state), 1.0)
print(f"Very small values (1e-15 to 1e-18): norm = {np.linalg.norm(state):.10f} ✓")

Very small values (1e-15 to 1e-18): norm = 1.0000000000 ✓


In [110]:
# Very large values — np.hypot prevents overflow
x_large = np.array([1e5, 1e5, 1e5, 1e5])
angles = enc.compute_encoding_angles(x_large)
print(f"Large values (1e5): equivariant angles = {angles['equivariant']}")
assert all(np.isfinite(angles['equivariant']))
print("All angles are finite ✓")

Large values (1e5): equivariant angles = [141421.35623731 141421.35623731 141421.35623731 141421.35623731]
All angles are finite ✓


In [111]:
# Zero input produces valid (finite) state
x_zeros = np.array([0.0, 0.0, 0.0, 0.0])
angles = enc.compute_encoding_angles(x_zeros)
print(f"Zero input: encoding angles    = {angles['encoding']}")
print(f"Zero input: equivariant angles = {angles['equivariant']}")
print(f"Zero input: interaction angles = {angles['interaction']}")
assert all(np.isfinite(angles['encoding']))
assert all(np.isfinite(angles['equivariant']))
print("All zero-input angles are finite ✓")

Zero input: encoding angles    = [0. 0. 0. 0.]
Zero input: equivariant angles = [0. 0. 0. 0.]
Zero input: interaction angles = [0. 0. 0.]
All zero-input angles are finite ✓


In [112]:
# Numerical stability across all symmetry types with random large values
np.random.seed(42)
x_mixed = np.array([1e-10, 1e5, -1e3, 0.5])

print("=== All symmetries with mixed-magnitude input ===")
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    n = 4 if sym != 'rotation' else 4
    enc = SymmetryInspiredFeatureMap(n_features=n, symmetry=sym, reps=1)
    # Use a valid input for each symmetry
    if sym == 'rotation':
        x_test = np.array([1e-10, 1e5, -1e3, 0.5])
    else:
        x_test = x_mixed[:n]
    angles = enc.compute_encoding_angles(x_test)
    all_finite = (all(np.isfinite(angles['encoding'])) and
                  all(np.isfinite(angles['equivariant'])))
    print(f"  {sym:12s}: all finite = {all_finite} ✓")

=== All symmetries with mixed-magnitude input ===
  rotation    : all finite = True ✓
  cyclic      : all finite = True ✓
  reflection  : all finite = True ✓
  full        : all finite = True ✓


---
## 33. Depth & Resource Scaling

Understanding how circuit resources scale with parameters helps in hardware planning.

In [113]:
# Depth scales linearly with reps
print("=== Depth vs reps (rotation, n=4, linear) ===")
for reps in [1, 2, 3, 5, 10]:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=reps)
    print(f"  reps={reps:2d}: depth={enc.depth:3d}, "
          f"gate_count={enc.properties.gate_count}")

=== Depth vs reps (rotation, n=4, linear) ===
  reps= 1: depth=  4, gate_count=20
  reps= 2: depth=  8, gate_count=40
  reps= 3: depth= 12, gate_count=60
  reps= 5: depth= 20, gate_count=100
  reps=10: depth= 40, gate_count=200


In [114]:
# Depth with different entanglement patterns (n_features=6)
print("=== Depth by entanglement (cyclic, n=6, reps=1) ===")
for ent in ['none', 'linear', 'circular', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=6, symmetry='cyclic', reps=1, entanglement=ent)
    n_pairs = len(enc.get_entanglement_pairs())
    print(f"  {ent:10s}: depth={enc.depth:3d}, pairs={n_pairs:3d}, "
          f"two_qubit={enc.properties.two_qubit_gates}")

=== Depth by entanglement (cyclic, n=6, reps=1) ===
  none      : depth=  3, pairs=  0, two_qubit=0
  linear    : depth=  5, pairs=  5, two_qubit=10
  circular  : depth=  5, pairs=  6, two_qubit=12
  full      : depth=  8, pairs= 15, two_qubit=30


In [115]:
# Gate count scaling with n_features
print("=== Gate count vs n_features (rotation, reps=2, linear) ===")
for n in [2, 4, 6, 8, 10]:
    enc = SymmetryInspiredFeatureMap(n_features=n, symmetry='rotation', reps=2)
    p = enc.properties
    print(f"  n={n:2d}: single={p.single_qubit_gates:3d}, "
          f"two_qubit={p.two_qubit_gates:3d}, total={p.gate_count:3d}")

=== Gate count vs n_features (rotation, reps=2, linear) ===
  n= 2: single= 16, two_qubit=  4, total= 20
  n= 4: single= 32, two_qubit=  8, total= 40
  n= 6: single= 48, two_qubit= 12, total= 60
  n= 8: single= 64, two_qubit= 16, total= 80
  n=10: single= 80, two_qubit= 20, total=100


---
## 34. Encoding Recommendation Guide

The library includes a recommendation system that suggests encodings based on problem characteristics.

In [116]:
from encoding_atlas.guide import recommend_encoding

# When might the guide recommend SymmetryInspired?
rec = recommend_encoding(
    n_features=4,
    n_samples=500,
    task='classification',
    hardware='simulator',
    priority='accuracy',
)
print(f"Recommendation for classification, accuracy priority:")
print(f"  Encoding:     {rec.encoding_name}")
print(f"  Confidence:   {rec.confidence:.2f}")
print(f"  Explanation:  {rec.explanation}")
print(f"  Alternatives: {rec.alternatives}")

Recommendation for classification, accuracy priority:
  Encoding:     iqp
  Confidence:   0.74
  Explanation:  IQP encoding creates highly entangled states with provable classical simulation hardness, well-suited for kernel methods
  Alternatives: ['data_reuploading', 'zz_feature_map', 'pauli_feature_map']


In [117]:
# Different priorities
print("=== Recommendations by priority (n=4, classification) ===")
for priority in ['accuracy', 'trainability', 'speed', 'noise_resilience']:
    rec = recommend_encoding(n_features=4, priority=priority)
    print(f"  {priority:18s}: {rec.encoding_name:30s} (confidence={rec.confidence:.2f})")

=== Recommendations by priority (n=4, classification) ===
  accuracy          : iqp                            (confidence=0.74)
  trainability      : data_reuploading               (confidence=0.56)
  speed             : angle                          (confidence=0.60)
  noise_resilience  : hardware_efficient             (confidence=0.60)


---
## 35. Visualization & Comparison

Compare SymmetryInspiredFeatureMap against other encodings.

In [118]:
from encoding_atlas.visualization import compare_encodings

# Text-based comparison
comparison_text = compare_encodings(
    encodings=["symmetry_inspired", "angle", "iqp", "zz_feature_map"],
    n_features=4,
    output="text",
)
print(comparison_text)

┌────────────────────────────────────────────────────────────────────────────┐
│                     ENCODING COMPARISON (n_features=4)                     │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                            │
│  QUBITS                            CIRCUIT DEPTH                           │
│  ──────                             ─────────────                          │
│  symmetry_inspired ███████████████ 4      symmetry_inspired █████          │
│  angle             ███████████████ 4      angle                            │
│  iqp               ███████████████ 4      iqp               ████           │
│  zz_feature_map    ███████████████ 4      zz_feature_map    ███████████████│
│                                                                            │
│  GATE COUNT                        TWO-QUBIT GATES                         │
│  ──────────                         ──────────────

---
## 36. Putting It All Together — Complete Workflow

A realistic end-to-end example: select, configure, analyze, and use a SymmetryInspiredFeatureMap for a quantum kernel classification task.

In [119]:
# Step 1: Choose and configure the encoding
enc = SymmetryInspiredFeatureMap(
    n_features=4,
    symmetry='rotation',
    reps=2,
    entanglement='linear',
    feature_map='angle',
)
print(f"Encoding: {enc}")
print(f"Properties: depth={enc.depth}, gates={enc.properties.gate_count}, "
      f"entangling={enc.properties.is_entangling}")

Encoding: SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2, entanglement='linear', feature_map='angle', include_barriers=True)
Properties: depth=8, gates=40, entangling=True


In [120]:
# Step 2: Analyze the encoding before using it
print("=== Pre-use Analysis ===")

# Simulability
sim_result = check_simulability(enc)
print(f"Simulability: {sim_result['simulability_class']}")

# Expressibility
expr = compute_expressibility(enc, n_samples=500, seed=42)
print(f"Expressibility: {expr:.4f}")

# Entanglement capability
ent = compute_entanglement_capability(enc, n_samples=200, seed=42)
print(f"Entanglement: {ent:.4f}")

# Trainability
train = estimate_trainability(enc, n_samples=100, seed=42)
print(f"Trainability: {train:.4f}")

# Resources
res = count_resources(enc)
print(f"Resources: {res['gate_count']} gates, {res['two_qubit_gates']} two-qubit")

=== Pre-use Analysis ===
Simulability: conditionally_simulable


c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=2.758, max=5.395). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=0.5917, max=6.13). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=0.805, max=5.823). This may produce unexpected rotation angl

Expressibility: 0.9214
Entanglement: 0.1307


c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=2.758, max=6.434). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 4 values outside range (min=4.328, max=5.395). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 3 values outside range (min=1.187, max=5.395). This may produce unexpected rotation angl

Trainability: 0.0924
Resources: 40 gates, 8 two-qubit


c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=0.0773, max=4.311). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 2 values outside range (min=0.0773, max=5.882). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  return _simulate_pennylane(encoding, x_array)
c:\Users\ashut\.venvs\ea-pypi\lib\site-packages\encoding_atlas\analysis\_utils.py:499: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=0.0773, max=3.696). This may produce unexpected rotation a

In [121]:
# Step 3: Prepare data and generate circuits
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

iris = load_iris()
X = iris.data[:, :4]  # 4 features
y = iris.target

# Scale to [0, pi] (recommended range for angle feature map)
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X)

# Generate circuits for all samples
circuits = enc.get_circuits(X_scaled, backend='pennylane', parallel=True)
print(f"Generated {len(circuits)} circuits for Iris dataset")

Generated 150 circuits for Iris dataset


C:\Users\ashut\AppData\Local\Programs\Python\Python310\lib\concurrent\futures\thread.py:58: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=2.094, max=3.142). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  result = self.fn(*self.args, **self.kwargs)
C:\Users\ashut\AppData\Local\Programs\Python\Python310\lib\concurrent\futures\thread.py:58: UserWarning: Input values outside recommended range [0, pi] for 'angle' feature mapping. Found 1 values outside range (min=1.702, max=3.142). This may produce unexpected rotation angles. Consider normalizing your data to [0, pi].
  result = self.fn(*self.args, **self.kwargs)


In [122]:
# Step 4: Compute quantum kernel
dev = qml.device("default.qubit", wires=enc.n_qubits)

@qml.qnode(dev)
def get_state_from_fn(fn):
    fn()
    return qml.state()

# Compute kernel for a small subset
n_subset = 20
states = []
for i in range(n_subset):
    state = get_state_from_fn(circuits[i])
    states.append(state)

# Kernel matrix: K(x, x') = |<psi(x)|psi(x')>|^2
kernel = np.zeros((n_subset, n_subset))
for i in range(n_subset):
    for j in range(i, n_subset):
        fid = np.abs(np.dot(np.conj(states[i]), states[j])) ** 2
        kernel[i, j] = fid
        kernel[j, i] = fid

print(f"Kernel matrix shape: {kernel.shape}")
print(f"Diagonal (self-fidelity): all 1.0 = {np.allclose(np.diag(kernel), 1.0)}")
print(f"Symmetric: {np.allclose(kernel, kernel.T)}")
print(f"Min off-diagonal: {kernel[np.triu_indices(n_subset, k=1)].min():.4f}")
print(f"Max off-diagonal: {kernel[np.triu_indices(n_subset, k=1)].max():.4f}")

Kernel matrix shape: (20, 20)
Diagonal (self-fidelity): all 1.0 = True
Symmetric: True
Min off-diagonal: 0.1723
Max off-diagonal: 0.9949


---
## 37. Distinguishability — Different Inputs Produce Different States

A key property of any useful encoding: distinct inputs should (generally) produce distinct quantum states.

In [123]:
enc = SymmetryInspiredFeatureMap(n_features=4, symmetry='rotation', reps=2)

x1 = np.array([0.1, 0.2, 0.3, 0.4])
x2 = np.array([0.5, 0.6, 0.7, 0.8])

state1 = simulate_encoding_statevector(enc, x1)
state2 = simulate_encoding_statevector(enc, x2)

fidelity = compute_fidelity(state1, state2)
print(f"Input 1: {x1}")
print(f"Input 2: {x2}")
print(f"Fidelity: {fidelity:.6f}")
assert fidelity < 1.0 - 1e-6, "Different inputs should produce different states"
print("Different inputs → different states ✓")

# Same input → same state (reproducibility)
state1_again = simulate_encoding_statevector(enc, x1)
fid_same = compute_fidelity(state1, state1_again)
assert np.isclose(fid_same, 1.0)
print(f"Same input → same state (fidelity = {fid_same:.10f}) ✓")

Input 1: [0.1 0.2 0.3 0.4]
Input 2: [0.5 0.6 0.7 0.8]
Fidelity: 0.642316
Different inputs → different states ✓
Same input → same state (fidelity = 1.0000000000) ✓


---
## 38. Entanglement Layer Information

The `_get_entanglement_layer_info()` private method (accessed via `get_layer_structure()`) shows the exact gates used in the entanglement layer for each symmetry type.

| Symmetry | Two-Qubit Gate | Additional Single-Qubit | Structure |
|---|---|---|---|
| `rotation` | CRZ | — | CRZ on coordinate pairs |
| `cyclic` | CNOT | RZ | CNOT + RZ + CNOT sandwich |
| `reflection` | CZ | RZ | CZ + parallel RZ on both qubits |
| `full` | CNOT | RY | CNOT + RY + CNOT + RY + CNOT permutation structure |

In [124]:
# Show entanglement structure per symmetry via layer structure
for sym in ['rotation', 'cyclic', 'reflection', 'full']:
    enc = SymmetryInspiredFeatureMap(n_features=4, symmetry=sym, reps=1)
    structure = enc.get_layer_structure()
    # Find the entanglement layer
    for layer in structure['layers']:
        if 'entanglement' in layer['name'].lower() or 'entangl' in layer['name'].lower():
            print(f"--- {sym} entanglement layer ---")
            print(f"  gate_types: {layer['gate_types']}")
            print(f"  n_gates:    {layer['n_gates']}")
            print(f"  {layer['description']}")
            print()
            break

--- rotation entanglement layer ---
  gate_types: ['CRZ']
  n_gates:    3
  rotation-inspired entanglement (rep 1)

--- cyclic entanglement layer ---
  gate_types: ['CNOT', 'RZ']
  n_gates:    9
  cyclic-inspired entanglement (rep 1)

--- reflection entanglement layer ---
  gate_types: ['CZ', 'RZ']
  n_gates:    9
  reflection-inspired entanglement (rep 1)

--- full entanglement layer ---
  gate_types: ['CNOT', 'RY']
  n_gates:    15
  full-inspired entanglement (rep 1)



---
## Summary

This notebook demonstrated every feature of `SymmetryInspiredFeatureMap` from the Quantum Encoding Atlas library:

### Constructor & Configuration
1. All 6 constructor parameters with defaults and valid ranges
2. All 4 symmetry types: rotation, cyclic, reflection, full
3. All 3 feature maps: angle, fourier, polynomial
4. All 4 entanglement patterns: none, linear, circular, full
5. Case-insensitive string parameters
6. Comprehensive validation (TypeError, ValueError) with informative messages
7. Behavioral warnings (rotation override, scalability)

### Properties & Inspection
8. Core properties: n_features, n_qubits, depth, symmetry, reps, entanglement, feature_map
9. EncodingProperties frozen dataclass (lazy, thread-safe, cached)
10. Config defensive copy
11. `get_symmetry_info()` — symmetry configuration details
12. `get_layer_structure()` — per-layer circuit breakdown
13. `gate_count_breakdown()` — TypedDict with per-gate-type counts
14. `resource_summary()` — comprehensive resource + hardware requirements
15. `compute_encoding_angles()` — debugging tool for angle computation

### Circuit Generation
16. PennyLane backend (callable closures)
17. Qiskit backend (QuantumCircuit with barriers)
18. Cirq backend (Moment-optimized circuits)
19. Cross-backend state equivalence
20. `convert_state_vector_ordering()` for qubit convention conversion

### Batch Processing
21. `get_circuits()` with sequential and parallel modes
22. Order preservation in parallel mode
23. Custom max_workers
24. 1D input auto-reshape

### Input Handling
25. Shape, value, type, and backend validation
26. NaN/Inf/complex rejection
27. List, tuple, integer array acceptance
28. Defensive copy isolation

### Analysis Tools
29. Simulability analysis (product states vs entangling)
30. Expressibility (KL divergence from Haar distribution)
31. Entanglement capability (Meyer-Wallach measure)
32. Trainability estimation (gradient variance)
33. Built-in `measure_trainability()` with local/global observables
34. Resource counting and comparison
35. Execution time estimation

### Advanced Features
36. Low-level utilities (statevector, fidelity, partial trace, entropy)
37. Capability protocols (ResourceAnalyzable, EntanglementQueryable)
38. Registry system (4 registered names)
39. Deprecated CovariantFeatureMap alias
40. Equality, hashing, set/dict usage
41. Pickle serialization with property recomputation
42. Thread-safe concurrent access
43. Logging & debugging
44. Numerical stability with extreme values

### Practical Usage
45. Encoding recommendation guide
46. Visualization & comparison
47. Complete quantum kernel workflow with Iris dataset
48. Input distinguishability verification